# 1. Connect the colab into Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
FOLDERNAME = 'Fighting_For_Thesis/Data/Running_simulating_model/'
assert FOLDERNAME is not None, "[!] Enter the foldername."
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd drive/My\ Drive/$FOLDERNAME/

# 2. Import library

## 2.1 Foudamental Libs

In [ ]:
import pandas as pd
import csv
import tensorflow.keras as keras
from numpy import dstack, hstack, mean, std
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, GRU, SimpleRNN, Bidirectional, Input, BatchNormalization, ConvLSTM2D, Conv1D, Conv2D, Conv3D,  MaxPooling1D, AveragePooling1D,MaxPooling2D, MaxPooling3D, concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D, TimeDistributed
from  tensorflow.keras.optimizers import SGD, Adam, RMSprop
from  tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import time
import itertools
import tensorflow.keras.layers as  Layer
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import backend as K
# from keras_multi_head import MultiHead
# from attention import Attention
# from keras_self_attention import SeqSelfAttention

## 2.2 Install sequence-self-attention

[keras-self-attention](https://github.com/CyberZHG/keras-self-attention)


In [ ]:
import tensorflow.keras
import tensorflow.keras.backend as K


class SeqSelfAttention(tensorflow.keras.layers.Layer):

    ATTENTION_TYPE_ADD = 'additive'
    ATTENTION_TYPE_MUL = 'multiplicative'

    def __init__(self, units=32, attention_width=None, attention_type=ATTENTION_TYPE_ADD, return_attention=False, history_only=False, kernel_initializer='glorot_normal', bias_initializer='zeros',kernel_regularizer=None,
                 bias_regularizer=None, kernel_constraint=None, bias_constraint=None, use_additive_bias=True,use_attention_bias=True,attention_activation=None,attention_regularizer_weight=0.0, **kwargs):
        """Layer initialization.
        For additive attention, see: https://arxiv.org/pdf/1806.01264.pdf
        :param units: The dimension of the vectors that used to calculate the attention weights.
        :param attention_width: The width of local attention.
        :param attention_type: 'additive' or 'multiplicative'.
        :param return_attention: Whether to return the attention weights for visualization.
        :param history_only: Only use historical pieces of data.
        :param kernel_initializer: The initializer for weight matrices.
        :param bias_initializer: The initializer for biases.
        :param kernel_regularizer: The regularization for weight matrices.
        :param bias_regularizer: The regularization for biases.
        :param kernel_constraint: The constraint for weight matrices.
        :param bias_constraint: The constraint for biases.
        :param use_additive_bias: Whether to use bias while calculating the relevance of inputs features
                                  in additive mode.
        :param use_attention_bias: Whether to use bias while calculating the weights of attention.
        :param attention_activation: The activation used for calculating the weights of attention.
        :param attention_regularizer_weight: The weights of attention regularizer.
        :param kwargs: Parameters for parent class.
        """
        super(SeqSelfAttention, self).__init__(**kwargs)
        self.supports_masking = True
        self.units = units
        self.attention_width = attention_width
        self.attention_type = attention_type
        self.return_attention = return_attention
        self.history_only = history_only
        if history_only and attention_width is None:
            self.attention_width = int(1e9)

        self.use_additive_bias = use_additive_bias
        self.use_attention_bias = use_attention_bias
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)
        self.kernel_constraint = keras.constraints.get(kernel_constraint)
        self.bias_constraint = keras.constraints.get(bias_constraint)
        self.attention_activation = keras.activations.get(attention_activation)
        self.attention_regularizer_weight = attention_regularizer_weight
        self._backend = keras.backend.backend()

        if attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self.Wx, self.Wt, self.bh = None, None, None
            self.Wa, self.ba = None, None
        elif attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self.Wa, self.ba = None, None
        else:
            raise NotImplementedError('No implementation for attention type : ' + attention_type)

    def get_config(self):
        config = {
            'units': self.units,
            'attention_width': self.attention_width,
            'attention_type': self.attention_type,
            'return_attention': self.return_attention,
            'history_only': self.history_only,
            'use_additive_bias': self.use_additive_bias,
            'use_attention_bias': self.use_attention_bias,
            'kernel_initializer': keras.initializers.serialize(self.kernel_initializer),
            'bias_initializer': keras.initializers.serialize(self.bias_initializer),
            'kernel_regularizer': keras.regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': keras.regularizers.serialize(self.bias_regularizer),
            'kernel_constraint': keras.constraints.serialize(self.kernel_constraint),
            'bias_constraint': keras.constraints.serialize(self.bias_constraint),
            'attention_activation': keras.activations.serialize(self.attention_activation),
            'attention_regularizer_weight': self.attention_regularizer_weight,
        }
        base_config = super(SeqSelfAttention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            self._build_additive_attention(input_shape)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            self._build_multiplicative_attention(input_shape)
        super(SeqSelfAttention, self).build(input_shape)

    def _build_additive_attention(self, input_shape):
        feature_dim = int(input_shape[2])

        self.Wt = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wt'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        self.Wx = self.add_weight(shape=(feature_dim, self.units),
                                  name='{}_Add_Wx'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_additive_bias:
            self.bh = self.add_weight(shape=(self.units,),
                                      name='{}_Add_bh'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

        self.Wa = self.add_weight(shape=(self.units, 1),
                                  name='{}_Add_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Add_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

    def _build_multiplicative_attention(self, input_shape):
        feature_dim = int(input_shape[2])

        self.Wa = self.add_weight(shape=(feature_dim, feature_dim),
                                  name='{}_Mul_Wa'.format(self.name),
                                  initializer=self.kernel_initializer,
                                  regularizer=self.kernel_regularizer,
                                  constraint=self.kernel_constraint)
        if self.use_attention_bias:
            self.ba = self.add_weight(shape=(1,),
                                      name='{}_Mul_ba'.format(self.name),
                                      initializer=self.bias_initializer,
                                      regularizer=self.bias_regularizer,
                                      constraint=self.bias_constraint)

    def call(self, inputs, mask=None, **kwargs):
        input_len = K.shape(inputs)[1]

        if self.attention_type == SeqSelfAttention.ATTENTION_TYPE_ADD:
            e = self._call_additive_emission(inputs)
        elif self.attention_type == SeqSelfAttention.ATTENTION_TYPE_MUL:
            e = self._call_multiplicative_emission(inputs)

        if self.attention_activation is not None:
            e = self.attention_activation(e)
        if self.attention_width is not None:
            if self.history_only:
                lower = K.arange(0, input_len) - (self.attention_width - 1)
            else:
                lower = K.arange(0, input_len) - self.attention_width // 2
            lower = K.expand_dims(lower, axis=-1)
            upper = lower + self.attention_width
            indices = K.expand_dims(K.arange(0, input_len), axis=0)
            e -= 10000.0 * (1.0 - K.cast(lower <= indices, K.floatx()) * K.cast(indices < upper, K.floatx()))
        if mask is not None:
            mask = K.expand_dims(K.cast(mask, K.floatx()), axis=-1)
            e -= 10000.0 * ((1.0 - mask) * (1.0 - K.permute_dimensions(mask, (0, 2, 1))))

        # a_{t} = \text{softmax}(e_t)
        e = K.exp(e - K.max(e, axis=-1, keepdims=True))
        a = e / K.sum(e, axis=-1, keepdims=True)

        # l_t = \sum_{t'} a_{t, t'} x_{t'}
        v = K.batch_dot(a, inputs)
        if self.attention_regularizer_weight > 0.0:
            self.add_loss(self._attention_regularizer(a))

        if self.return_attention:
            return [v, a]
        return v

    def _call_additive_emission(self, inputs):
        input_shape = K.shape(inputs)
        batch_size, input_len = input_shape[0], input_shape[1]

        # h_{t, t'} = \tanh(x_t^T W_t + x_{t'}^T W_x + b_h)
        q = K.expand_dims(K.dot(inputs, self.Wt), 2)
        k = K.expand_dims(K.dot(inputs, self.Wx), 1)
        if self.use_additive_bias:
            h = K.tanh(q + k + self.bh)
        else:
            h = K.tanh(q + k)

        # e_{t, t'} = W_a h_{t, t'} + b_a
        if self.use_attention_bias:
            e = K.reshape(K.dot(h, self.Wa) + self.ba, (batch_size, input_len, input_len))
        else:
            e = K.reshape(K.dot(h, self.Wa), (batch_size, input_len, input_len))
        return e

    def _call_multiplicative_emission(self, inputs):
        # e_{t, t'} = x_t^T W_a x_{t'} + b_a
        e = K.batch_dot(K.dot(inputs, self.Wa), K.permute_dimensions(inputs, (0, 2, 1)))
        if self.use_attention_bias:
            e += self.ba[0]
        return e

    def compute_output_shape(self, input_shape):
        output_shape = input_shape
        if self.return_attention:
            attention_shape = (input_shape[0], output_shape[1], input_shape[1])
            return [output_shape, attention_shape]
        return output_shape

    def compute_mask(self, inputs, mask=None):
        if self.return_attention:
            return [mask, None]
        return mask

    def _attention_regularizer(self, attention):
        batch_size = K.cast(K.shape(attention)[0], K.floatx())
        input_len = K.shape(attention)[-1]
        indices = K.expand_dims(K.arange(0, input_len), axis=0)
        diagonal = K.expand_dims(K.arange(0, input_len), axis=-1)
        eye = K.cast(K.equal(indices, diagonal), K.floatx())
        return self.attention_regularizer_weight * K.sum(K.square(K.batch_dot(
            attention,
            K.permute_dimensions(attention, (0, 2, 1))) - eye)) / batch_size

    @staticmethod
    def get_custom_objects():
        return {'SeqSelfAttention': SeqSelfAttention}

## 2.3. Extra Functions - get_trainable_params  and plot_confusion_matrix

In [ ]:

def get_trainable_params(model):
  # this quickly gets weights for each layer
  number_of_weights = [K.count_params(w) for w in model.trainable_weights]
  # loop all layers for more details
  k = 0
  for layer in model.layers:
    print('_ _ _ '*10)
    print(layer.name)
    for weights in layer.trainable_weights:
      print(f'{weights.name}, \nshape={weights.shape} ===> {number_of_weights[k]}')
      k+=1
  print('_ _ _ '*10)
  total = np.array(number_of_weights).sum().astype(int)
  print(f'Total = {total}')
  return total
  
  
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues,figsize=(5,5)):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    adapted from https://scikit-learn.org/0.18/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
    else:
      print('Confusion matrix, without normalization')
    
    print(cm)
    
    f, ax = plt.subplots(1,figsize=figsize)
    ax.imshow(cm, interpolation='nearest', cmap=cmap,)
    ax.set_title(title)
    # ax.setcolorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      ax.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")
    
    f.tight_layout()
    ax.set_ylabel('True label')
    ax.set_xlabel('Predicted label')

# 3. Training and Testing data && function dataLoader

## 3.1 Cold-Start

### 3.11 traning_data

In [ ]:

data_filenames = [
          'Training_data/0/data_0_22_v7.csv', 'Training_data/0/data_0_22_2_v7.csv', 'Training_data/0/data_0_121_s1_t1_1_v7.csv', 'Training_data/0/data_0_211_s1_t1_1_v7.csv', 'Training_data/0/data_0_220_s1_t1_1_v7.csv', 'Training_data/0/data_0_221_s1_t1_1_v7.csv',
          'Training_data/0/data_0_221_s1_t2_1_v7.csv', 'Training_data/0/data_0_221_s1_t3_1_v7.csv', 'Training_data/0/data_0_221_s1_t4_3_v7.csv', 'Training_data/0/data_0_221_s1_t6_3_v7.csv', 'Training_data/0/data_0_221_s2_t1_1_v7.csv', 'Training_data/0/data_0_221_s3_t1_1_v7.csv',
          'Training_data/0/data_0_221_s3_t4_1_v7.csv', 'Training_data/0/data_0_221_s3_t5_1_v7.csv', 'Training_data/0/data_0_221_s4_t1_1_v7.csv','Training_data/0/data_0_221_s5_t1_1_v7.csv', 'Training_data/0/data_0_222_s1_t1_1_v7.csv', 'Training_data/0/data_0_222_s1_t2_1_v7.csv', 
          'Training_data/0/data_0_221_s2_t1_1_v7.csv',  

          'Training_data/1/data_1_011_021_v7.csv', 'Training_data/1/data_1_120_122_v7.csv', 'Training_data/1/data_1_021_s1_t1_1_v7.csv', 'Training_data/1/data_1_120_s1_t1_1_v7.csv','Training_data/1/data_1_122_s1_t1_1_v7.csv', 'Training_data/1/data_1_021_s2_t1_1_v7.csv',
          'Training_data/1/data_1_221_s3_t1_1_v7.csv', 'Training_data/1/data_1_221_s4_t1_1_v7.csv', 'Training_data/1/data_1_221_s5_t1_1_v7.csv', 'Training_data/1/data_1_221_s6_t1_1_v7.csv', 'Training_data/1/data_1_221_s3_t1_2_v7.csv', 'Training_data/1/data_1_221_s4_t1_3_v7.csv',
          'Training_data/1/data_1_222_s1_t1_1_v7.csv', 'Training_data/1/data_1_222_s1_t2_1_v7.csv', 'Training_data/1/data_1_222_s2_t1_1_v7.csv',

          'Training_data/2/data_2_22_v7.csv', 'Training_data/2/data_2_22_2_v7.csv', 'Training_data/2/data_2_121_s1_t1_1_v7.csv', 'Training_data/2/data_2_211_s1_t1_1_v7.csv', 'Training_data/2/data_2_220_s1_t1_1_v7.csv','Training_data/2/data_2_221_s1_t1_1_v7.csv',
          'Training_data/2/data_2_222_s1_t1_1_v7.csv', 'Training_data/2/data_2_221_s2_t1_1_v7.csv', 'Training_data/2/data_2_221_s3_t1_1_v7.csv', 'Training_data/2/data_2_221_s3_t1_3_v7.csv','Training_data/2/data_2_222_s2_t1_1_v7.csv', 'Training_data/2/data_2_222_s2_t2_1_v7.csv',
          'Training_data/2/data_2_222_s3_t1_1_v7.csv',

          'Training_data/3/data_3_22_v7.csv','Training_data/3/data_3_22_2_v7.csv','Training_data/3/data_3_121_s1_t1_1_v7.csv','Training_data/3/data_3_121_s2_t1_1_v7.csv', 'Training_data/3/data_3_220_s1_t1_1_v7.csv','Training_data/3/data_3_220_s2_t1_1_v7.csv',
          'Training_data/3/data_3_221_s1_t1_1_v7.csv','Training_data/3/data_3_221_s2_t1_1_v7.csv', 'Training_data/3/data_3_222_s1_t1_1_v7.csv','Training_data/3/data_3_222_s1_t2_1_v7.csv', 'Training_data/3/data_3_222_s2_t1_1_v7.csv','Training_data/3/data_3_222_s2_t2_1_v7.csv',  
                  
          'Training_data/4/data_4_221_v7.csv', 'Training_data/4/data_4_221_2_v7.csv','Training_data/4/data_4_121_s1_t1_1_v7.csv','Training_data/4/data_4_121_s2_t1_1_v7.csv', 'Training_data/4/data_4_211_s1_t1_1_v7.csv', 'Training_data/4/data_4_211_s2_t1_1_v7.csv', 
          'Training_data/4/data_4_210_s1_t1_1_v7.csv','Training_data/4/data_4_210_s2_t1_1_v7.csv','Training_data/4/data_4_220_s1_t1_1_v7.csv','Training_data/4/data_4_220_s2_t1_1_v7.csv','Training_data/4/data_4_221_s1_t1_1_v7.csv','Training_data/4/data_4_221_s2_t1_1_v7.csv',
          'Training_data/4/data_4_221_s2_t2_1_v7.csv','Training_data/4/data_4_221_s2_t1_3_v7.csv','Training_data/4/data_4_221_s2_t2_2_v7.csv','Training_data/4/data_4_221_s3_t1_1_v7.csv','Training_data/4/data_4_221_s4_t1_1_v7.csv','Training_data/4/data_4_221_s5_t1_1_v7.csv',
          'Training_data/4/data_4_221_s6_t1_1_v7.csv','Training_data/4/data_4_221_s6_t1_2_v7.csv','Training_data/4/data_4_221_s6_t1_3_v7.csv','Training_data/4/data_4_222_s1_t1_1_v7.csv','Training_data/4/data_4_222_s1_t2_1_v7.csv','Training_data/4/data_4_222_s2_t1_1_v7.csv',

          'Training_data/5/data_5_221_v7.csv', 'Training_data/5/data_5_221_2_v7.csv', 'Training_data/5/data_5_121_s1_t1_1_v7.csv','Training_data/5/data_5_121_s1_t2_1_v7.csv','Training_data/5/data_5_220_s1_t1_1_v7.csv','Training_data/5/data_5_220_s1_t2_1_v7.csv',
          'Training_data/5/data_5_221_s1_t1_1_v7.csv','Training_data/5/data_5_221_s1_t2_1_v7.csv','Training_data/5/data_5_221_s1_t3_1_v7.csv','Training_data/5/data_5_221_s1_t3_3_v7.csv','Training_data/5/data_5_221_s2_t2_3_v7.csv','Training_data/5/data_5_221_s3_t2_1_v7.csv',
          'Training_data/5/data_5_221_s3_t3_1_v7.csv','Training_data/5/data_5_221_s3_t3_2_v7.csv','Training_data/5/data_5_222_s1_t1_1_v7.csv','Training_data/5/data_5_222_s1_t2_1_v7.csv','Training_data/5/data_5_222_s2_t1_1_v7.csv','Training_data/5/data_5_222_s2_t2_1_v7.csv',

          'Training_data/6/data_6_121_v7.csv', 'Training_data/6/data_6_221_220_v7.csv', 'Training_data/6/data_6_121_s1_t1_1_v7.csv','Training_data/6/data_6_221_s1_t1_1_v7.csv','Training_data/6/data_6_221_s2_t1_1_v7.csv','Training_data/6/data_6_221_s3_t1_1_v7.csv',
          'Training_data/6/data_6_221_s3_t1_3_v7.csv','Training_data/6/data_6_221_s3_t2_1_v7.csv','Training_data/6/data_6_221_s3_t2_2_v7.csv','Training_data/6/data_6_221_s3_t2_3_v7.csv','Training_data/6/data_6_222_s1_t1_1_v7.csv','Training_data/6/data_6_222_s1_t2_1_v7.csv',
          'Training_data/6/data_6_222_s2_t1_1_v7.csv','Training_data/6/data_6_222_s2_t2_1_v7.csv',
                  
          'Training_data/7/data_7_221_v7.csv','Training_data/7/data_7_221_3_v7.csv', 'Training_data/7/data_7_221_2_v7.csv', 'Training_data/7/data_7_121_s1_t1_1_v7.csv','Training_data/7/data_7_121_s2_t1_1_v7.csv','Training_data/7/data_7_121_s3_t1_1_v7.csv', 
          'Training_data/7/data_7_221_s1_t1_1_v7.csv','Training_data/7/data_7_221_s2_t1_1_v7.csv','Training_data/7/data_7_221_s3_t1_1_v7.csv','Training_data/7/data_7_221_s4_t1_1_v7.csv','Training_data/7/data_7_221_s5_t1_1_v7.csv','Training_data/7/data_7_221_s5_t1_3_v7.csv',
          'Training_data/7/data_7_221_s6_t1_1_v7.csv','Training_data/7/data_7_221_s7_t1_1_v7.csv','Training_data/7/data_7_221_s7_t1_3_v7.csv','Training_data/7/data_7_222_s1_t1_1_v7.csv','Training_data/7/data_7_222_s1_t2_1_v7.csv','Training_data/7/data_7_222_s2_t1_1_v7.csv',
          'Training_data/7/data_7_222_s2_t2_1_v7.csv',   
                  
          'Training_data/8/data_8_221_v7.csv', 'Training_data/8/data_8_221_3_v7.csv', 'Training_data/8/data_8_121_s1_t1_1_v7.csv','Training_data/8/data_8_121_s1_t2_1_v7.csv','Training_data/8/data_8_121_s1_t3_1_v7.csv', 'Training_data/8/data_8_220_s1_t1_1_v7.csv',
          'Training_data/8/data_8_220_s1_t2_1_v7.csv','Training_data/8/data_8_220_s1_t3_1_v7.csv','Training_data/8/data_8_221_s1_t1_1_v7.csv','Training_data/8/data_8_221_s1_t2_1_v7.csv','Training_data/8/data_8_221_s1_t3_1_v7.csv','Training_data/8/data_8_221_s1_t4_1_v7.csv',
          'Training_data/8/data_8_221_s1_t5_1_v7.csv','Training_data/8/data_8_121_s1_t4_1_v7.csv','Training_data/8/data_8_221_s1_t6_1_v7.csv','Training_data/8/data_8_221_s1_t7_1_v7.csv','Training_data/8/data_8_221_s1_t7_2_v7.csv','Training_data/8/data_8_221_s1_t7_3_v7.csv',
          'Training_data/8/data_8_221_s1_t8_1_v7.csv','Training_data/8/data_8_221_s1_t8_3_v7.csv','Training_data/8/data_8_221_s2_t1_1_v7.csv','Training_data/8/data_8_222_s1_t1_1_v7.csv','Training_data/8/data_8_222_s1_t2_1_v7.csv','Training_data/8/data_8_222_s2_t1_1_v7.csv',
                  
          'Training_data/9/data_9_120_v7.csv', 'Training_data/9/data_9_221_220_v7.csv','Training_data/9/data_9_221_220_3_v7.csv','Training_data/9/data_9_120_s1_t1_1_v7.csv','Training_data/9/data_9_121_s1_t1_1_v7.csv','Training_data/9/data_9_220_s1_t1_1_v7.csv',
          'Training_data/9/data_9_221_s1_t1_1_v7.csv','Training_data/9/data_9_220_s1_t2_1_v7.csv','Training_data/9/data_9_221_s1_t2_2_v7.csv','Training_data/9/data_9_221_s1_t2_3_v7.csv','Training_data/9/data_9_221_s2_t1_1_v7.csv','Training_data/9/data_9_221_s3_t1_1_v7.csv',
          'Training_data/9/data_9_221_s3_t1_2_v7.csv' ,'Training_data/9/data_9_221_s3_t1_3_v7.csv' ,'Training_data/9/data_9_221_s4_t1_3_v7.csv','Training_data/9/data_9_222_s1_t1_1_v7.csv','Training_data/9/data_9_222_s1_t2_1_v7.csv','Training_data/9/data_9_222_s2_t1_1_v7.csv',
                   
          ]


label_filenames = [
'Training_data/0/data_0_22_v7.csv_label_v1.txt','Training_data/0/data_0_22_2_v7.csv_label_v1.txt','Training_data/0/data_0_121_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_211_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_220_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t1_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/0/data_0_221_s1_t3_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t4_3_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t6_3_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s2_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s3_t1_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s3_t4_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s3_t5_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/0/data_0_221_s5_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_222_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_222_s1_t2_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s2_t1_1_v7.csv_label_v1.txt', 

'Training_data/1/data_1_011_021_v7.csv_label_v1.txt','Training_data/1/data_1_120_122_v7.csv_label_v1.txt', 'Training_data/1/data_1_021_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_120_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_122_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_021_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/1/data_1_221_s3_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s5_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s6_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s3_t1_2_v7.csv_label_v1.txt','Training_data/1/data_1_221_s4_t1_3_v7.csv_label_v1.txt',
'Training_data/1/data_1_222_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/1/data_1_222_s1_t2_1_v7.csv_label_v1.txt', 'Training_data/1/data_1_222_s2_t1_1_v7.csv_label_v1.txt',

'Training_data/2/data_2_22_v7.csv_label_v1.txt','Training_data/2/data_2_22_2_v7.csv_label_v1.txt','Training_data/2/data_2_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_211_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/2/data_2_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s3_t1_1_v7.csv_label_v1.txt', 'Training_data/2/data_2_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/2/data_2_222_s2_t1_1_v7.csv_label_v1.txt', 'Training_data/2/data_2_222_s2_t2_1_v7.csv_label_v1.txt',
'Training_data/2/data_2_222_s3_t1_1_v7.csv_label_v1.txt', 
                  
'Training_data/3/data_3_22_v7.csv_label_v1.txt','Training_data/3/data_3_22_2_v7.csv_label_v1.txt','Training_data/3/data_3_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_220_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/3/data_3_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s1_t2_1_v7.csv_label_v1.txt', 'Training_data/3/data_3_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s2_t2_1_v7.csv_label_v1.txt',  
                  
'Training_data/4/data_4_221_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_2_v7.csv_label_v1.txt','Training_data/4/data_4_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_211_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/4/data_4_211_s2_t1_1_v7.csv_label_v1.txt', 
'Training_data/4/data_4_210_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_210_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_220_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/4/data_4_221_s2_t2_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t1_3_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t2_2_v7.csv_label_v1.txt','Training_data/4/data_4_221_s3_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s5_t1_1_v7.csv_label_v1.txt',
'Training_data/4/data_4_221_s6_t1_1_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_s6_t1_2_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_s6_t1_3_v7.csv_label_v1.txt','Training_data/4/data_4_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/4/data_4_222_s2_t1_1_v7.csv_label_v1.txt', 
                  
'Training_data/5/data_5_221_v7.csv_label_v1.txt', 'Training_data/5/data_5_221_2_v7.csv_label_v1.txt', 'Training_data/5/data_5_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_121_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_220_s1_t2_1_v7.csv_label_v1.txt',
'Training_data/5/data_5_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t3_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t3_3_v7.csv_label_v1.txt','Training_data/5/data_5_221_s2_t2_3_v7.csv_label_v1.txt','Training_data/5/data_5_221_s3_t2_1_v7.csv_label_v1.txt',
'Training_data/5/data_5_221_s3_t3_1_v7.csv_label_v1.txt', 'Training_data/5/data_5_221_s3_t3_2_v7.csv_label_v1.txt','Training_data/5/data_5_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s2_t2_1_v7.csv_label_v1.txt', 
                  
'Training_data/6/data_6_121_v7.csv_label_v1.txt', 'Training_data/6/data_6_221_220_v7.csv_label_v1.txt', 'Training_data/6/data_6_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t1_1_v7.csv_label_v1.txt',
'Training_data/6/data_6_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_2_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_3_v7.csv_label_v1.txt','Training_data/6/data_6_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_222_s1_t2_1_v7.csv_label_v1.txt',
'Training_data/6/data_6_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_222_s2_t2_1_v7.csv_label_v1.txt',
                  
'Training_data/7/data_7_221_v7.csv_label_v1.txt','Training_data/7/data_7_221_3_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_2_v7.csv_label_v1.txt', 'Training_data/7/data_7_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_121_s3_t1_1_v7.csv_label_v1.txt', 
'Training_data/7/data_7_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_221_s3_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s4_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s5_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s5_t1_3_v7.csv_label_v1.txt', 
'Training_data/7/data_7_221_s6_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s7_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s7_t1_3_v7.csv_label_v1.txt','Training_data/7/data_7_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/7/data_7_222_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/7/data_7_222_s2_t2_1_v7.csv_label_v1.txt',   
                  
'Training_data/8/data_8_221_v7.csv_label_v1.txt', 'Training_data/8/data_8_221_3_v7.csv_label_v1.txt', 'Training_data/8/data_8_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t3_1_v7.csv_label_v1.txt', 'Training_data/8/data_8_220_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/8/data_8_220_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_220_s1_t3_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t3_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t4_1_v7.csv_label_v1.txt',
'Training_data/8/data_8_221_s1_t5_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t4_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t6_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_2_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_3_v7.csv_label_v1.txt',
'Training_data/8/data_8_221_s1_t8_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t8_3_v7.csv_label_v1.txt','Training_data/8/data_8_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s2_t1_1_v7.csv_label_v1.txt',
                  
'Training_data/9/data_9_120_v7.csv_label_v1.txt', 'Training_data/9/data_9_221_220_v7.csv_label_v1.txt','Training_data/9/data_9_221_220_3_v7.csv_label_v1.txt','Training_data/9/data_9_120_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_220_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/9/data_9_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_220_s1_t2_1_v7.csv_label_v1.txt','Training_data/9/data_9_221_s1_t2_2_v7.csv_label_v1.txt','Training_data/9/data_9_221_s1_t2_3_v7.csv_label_v1.txt','Training_data/9/data_9_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_221_s3_t1_1_v7.csv_label_v1.txt',
'Training_data/9/data_9_221_s3_t1_2_v7.csv_label_v1.txt','Training_data/9/data_9_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/9/data_9_221_s4_t1_3_v7.csv_label_v1.txt','Training_data/9/data_9_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/9/data_9_222_s2_t1_1_v7.csv_label_v1.txt',
                
                              
          ]
   

### 3.12 testing_data

In [ ]:
data_test_filenames_new = [
'Testing_data/HaozheTang/data_0_HaozheTang_v7.csv','Testing_data/HaozheTang/data_1_HaozheTang_v7.csv','Testing_data/HaozheTang/data_2_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_3_HaozheTang_v7.csv','Testing_data/HaozheTang/data_4_HaozheTang_v7.csv','Testing_data/HaozheTang/data_5_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_6_HaozheTang_v7.csv','Testing_data/HaozheTang/data_7_HaozheTang_v7.csv','Testing_data/HaozheTang/data_8_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_9_HaozheTang_v7.csv',
'Testing_data/Qing/data_0_Qing_v7.csv', 'Testing_data/Qing/data_1_Qing_v7.csv', 'Testing_data/Qing/data_2_Qing_v7.csv',
'Testing_data/Qing/data_3_Qing_v7.csv', 'Testing_data/Qing/data_4_Qing_v7.csv', 'Testing_data/Qing/data_5_Qing_v7.csv',
'Testing_data/Qing/data_6_Qing_v7.csv', 'Testing_data/Qing/data_7_Qing_v7.csv', 'Testing_data/Qing/data_8_Qing_v7.csv',
'Testing_data/Qing/data_9_Qing_v7.csv',
'Testing_data/Serge/data_0_Serge_v7.csv','Testing_data/Serge/data_1_Serge_v7.csv','Testing_data/Serge/data_2_Serge_v7.csv',
'Testing_data/Serge/data_3_Serge_v7.csv','Testing_data/Serge/data_4_Serge_v7.csv','Testing_data/Serge/data_5_Serge_v7.csv',
'Testing_data/Serge/data_6_Serge_v7.csv','Testing_data/Serge/data_7_Serge_v7.csv','Testing_data/Serge/data_8_Serge_v7.csv',
'Testing_data/Serge/data_9_Serge_v7.csv',
'Testing_data/AlexCui/data_0_AlexCui_v7.csv','Testing_data/AlexCui/data_1_AlexCui_v7.csv','Testing_data/AlexCui/data_2_AlexCui_v7.csv',
'Testing_data/AlexCui/data_3_AlexCui_v7.csv','Testing_data/AlexCui/data_4_AlexCui_v7.csv','Testing_data/AlexCui/data_5_AlexCui_v7.csv',
'Testing_data/AlexCui/data_6_AlexCui_v7.csv','Testing_data/AlexCui/data_7_AlexCui_v7.csv','Testing_data/AlexCui/data_8_AlexCui_v7.csv',
'Testing_data/AlexCui/data_9_AlexCui_v7.csv', 
'Testing_data/Keyarash/data_0_Keyarash_v7.csv','Testing_data/Keyarash/data_1_Keyarash_v7.csv','Testing_data/Keyarash/data_2_Keyarash_v7.csv',
'Testing_data/Keyarash/data_3_Keyarash_v7.csv','Testing_data/Keyarash/data_4_Keyarash_v7.csv','Testing_data/Keyarash/data_5_Keyarash_v7.csv',
'Testing_data/Keyarash/data_6_Keyarash_v7.csv','Testing_data/Keyarash/data_7_Keyarash_v7.csv','Testing_data/Keyarash/data_8_Keyarash_v7.csv',
'Testing_data/Keyarash/data_9_Keyarash_v7.csv',
'Testing_data/PingweiJiao/data_0_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_1_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_2_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_4_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_5_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_7_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_8_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_9_PingweiJiao_v7.csv',                        
'Testing_data/Talia/data_1_Talia_v7.csv','Testing_data/Talia/data_2_Talia_v7.csv',
'Testing_data/Talia/data_3_Talia_v7.csv','Testing_data/Talia/data_4_Talia_v7.csv','Testing_data/Talia/data_5_Talia_v7.csv',
'Testing_data/Talia/data_7_Talia_v7.csv',
'Testing_data/Talia/data_9_Talia_v7.csv',
'Testing_data/Vineet/data_0_Vineet_v7.csv','Testing_data/Vineet/data_1_Vineet_v7.csv','Testing_data/Vineet/data_2_Vineet_v7.csv',
'Testing_data/Vineet/data_3_Vineet_v7.csv','Testing_data/Vineet/data_4_Vineet_v7.csv','Testing_data/Vineet/data_5_Vineet_v7.csv',
'Testing_data/Vineet/data_6_Vineet_v7.csv','Testing_data/Vineet/data_7_Vineet_v7.csv','Testing_data/Vineet/data_8_Vineet_v7.csv',
'Testing_data/Vineet/data_9_Vineet_v7.csv',   
'Testing_data/Song/data_0_Song_v7.csv','Testing_data/Song/data_1_Song_v7.csv','Testing_data/Song/data_2_Song_v7.csv',
'Testing_data/Song/data_3_Song_v7.csv','Testing_data/Song/data_4_Song_v7.csv','Testing_data/Song/data_5_Song_v7.csv',
'Testing_data/Song/data_6_Song_v7.csv','Testing_data/Song/data_7_Song_v7.csv','Testing_data/Song/data_8_Song_v7.csv',
'Testing_data/Song/data_9_Song_v7.csv',   
'Testing_data/Vito/data_0_Vito_v7.csv','Testing_data/Vito/data_1_Vito_v7.csv','Testing_data/Vito/data_2_Vito_v7.csv',
'Testing_data/Vito/data_3_Vito_v7.csv','Testing_data/Vito/data_4_Vito_v7.csv','Testing_data/Vito/data_5_Vito_v7.csv',
'Testing_data/Vito/data_6_Vito_v7.csv','Testing_data/Vito/data_7_Vito_v7.csv','Testing_data/Vito/data_8_Vito_v7.csv',
'Testing_data/Vito/data_9_Vito_v7.csv', 
'Testing_data/YangLi/data_0_YangLi_v7.csv','Testing_data/YangLi/data_1_YangLi_v7.csv','Testing_data/YangLi/data_2_YangLi_v7.csv',
'Testing_data/YangLi/data_3_YangLi_v7.csv','Testing_data/YangLi/data_4_YangLi_v7.csv','Testing_data/YangLi/data_5_YangLi_v7.csv',
'Testing_data/YangLi/data_6_YangLi_v7.csv','Testing_data/YangLi/data_7_YangLi_v7.csv',
'Testing_data/YangLi/data_9_YangLi_v7.csv',
'Testing_data/HanTing/data_0_HanTing_v7.csv','Testing_data/HanTing/data_1_HanTing_v7.csv','Testing_data/HanTing/data_2_HanTing_v7.csv',
'Testing_data/HanTing/data_3_HanTing_v7.csv','Testing_data/HanTing/data_4_HanTing_v7.csv','Testing_data/HanTing/data_5_HanTing_v7.csv',
'Testing_data/HanTing/data_6_HanTing_v7.csv','Testing_data/HanTing/data_7_HanTing_v7.csv','Testing_data/HanTing/data_8_HanTing_v7.csv',
'Testing_data/HanTing/data_9_HanTing_v7.csv',
'Testing_data/Naram/data_1_Naram_v7.csv','Testing_data/Naram/data_2_Naram_v7.csv','Testing_data/Naram/data_3_Naram_v7.csv',
'Testing_data/Naram/data_4_Naram_v7.csv','Testing_data/Naram/data_5_Naram_v7.csv','Testing_data/Naram/data_6_Naram_v7.csv',
'Testing_data/Naram/data_7_Naram_v7.csv','Testing_data/Naram/data_8_Naram_v7.csv','Testing_data/Naram/data_9_Naram_v7.csv',   
'Testing_data/Ricardo/data_0_Ricardo_v7.csv','Testing_data/Ricardo/data_1_Ricardo_v7.csv','Testing_data/Ricardo/data_2_Ricardo_v7.csv',
'Testing_data/Ricardo/data_3_Ricardo_v7.csv','Testing_data/Ricardo/data_4_Ricardo_v7.csv','Testing_data/Ricardo/data_5_Ricardo_v7.csv',
'Testing_data/Ricardo/data_6_Ricardo_v7.csv','Testing_data/Ricardo/data_7_Ricardo_v7.csv','Testing_data/Ricardo/data_8_Ricardo_v7.csv',
'Testing_data/Ricardo/data_9_Ricardo_v7.csv',
'Testing_data/Weichuan/data_0_Weichuan_v7.csv','Testing_data/Weichuan/data_1_Weichuan_v7.csv','Testing_data/Weichuan/data_2_Weichuan_v7.csv',
'Testing_data/Weichuan/data_3_Weichuan_v7.csv','Testing_data/Weichuan/data_4_Weichuan_v7.csv','Testing_data/Weichuan/data_5_Weichuan_v7.csv',
'Testing_data/Weichuan/data_6_Weichuan_v7.csv','Testing_data/Weichuan/data_7_Weichuan_v7.csv','Testing_data/Weichuan/data_8_Weichuan_v7.csv',       
'Testing_data/jundaWu/data_0_jundaWu_v7.csv','Testing_data/jundaWu/data_1_jundaWu_v7.csv','Testing_data/jundaWu/data_2_jundaWu_v7.csv',
'Testing_data/jundaWu/data_3_jundaWu_v7.csv','Testing_data/jundaWu/data_4_jundaWu_v7.csv','Testing_data/jundaWu/data_5_jundaWu_v7.csv',
'Testing_data/jundaWu/data_6_jundaWu_v7.csv','Testing_data/jundaWu/data_7_jundaWu_v7.csv','Testing_data/jundaWu/data_8_jundaWu_v7.csv',
'Testing_data/jundaWu/data_9_jundaWu_v7.csv',
'Testing_data/shixunWu/data_0_shiXun_v7.csv','Testing_data/shixunWu/data_1_shiXun_v7.csv','Testing_data/shixunWu/data_2_shiXun_v7.csv',
'Testing_data/shixunWu/data_3_shiXun_v7.csv','Testing_data/shixunWu/data_4_shiXun_v7.csv','Testing_data/shixunWu/data_5_shiXun_v7.csv',
'Testing_data/shixunWu/data_6_shiXun_v7.csv','Testing_data/shixunWu/data_7_shiXun_v7.csv','Testing_data/shixunWu/data_8_shiXun_v7.csv',
'Testing_data/shixunWu/data_9_shiXun_v7.csv',
'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv',
'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv',
'Testing_data/shuxinChen/data_0_shuxinChen_v7.csv','Testing_data/shuxinChen/data_1_shuxinChen_v7.csv','Testing_data/shuxinChen/data_2_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_3_shuxinChen_v7.csv','Testing_data/shuxinChen/data_4_shuxinChen_v7.csv','Testing_data/shuxinChen/data_5_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_6_shuxinChen_v7.csv','Testing_data/shuxinChen/data_7_shuxinChen_v7.csv','Testing_data/shuxinChen/data_8_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_9_shuxinChen_v7.csv',
'Testing_data/zehangWu/data_0_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_3_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_6_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_9_ZehangWu_v7.csv',
'Testing_data/liang/data_0_Liang_v7.csv','Testing_data/liang/data_1_Liang_v7.csv','Testing_data/liang/data_2_Liang_v7.csv',
'Testing_data/liang/data_3_Liang_v7.csv','Testing_data/liang/data_4_Liang_v7.csv','Testing_data/liang/data_5_Liang_v7.csv',
'Testing_data/liang/data_6_Liang_v7.csv','Testing_data/liang/data_7_Liang_v7.csv','Testing_data/liang/data_8_Liang_v7.csv',
'Testing_data/liang/data_9_Liang_v7.csv',
'Testing_data/TingLiang/data_0_TingLiang_v7.csv', 'Testing_data/TingLiang/data_1_TingLiang_v7.csv','Testing_data/TingLiang/data_2_TingLiang_v7.csv',
'Testing_data/TingLiang/data_3_TingLiang_v7.csv','Testing_data/TingLiang/data_4_TingLiang_v7.csv','Testing_data/TingLiang/data_5_TingLiang_v7.csv',
'Testing_data/TingLiang/data_6_TingLiang_v7.csv','Testing_data/TingLiang/data_7_TingLiang_v7.csv','Testing_data/TingLiang/data_8_TingLiang_v7.csv',
'Testing_data/TingLiang/data_9_TingLiang_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_0_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_1_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_2_ZehangWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_4_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_5_ZehangWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_7_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_8_ZehangWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_ZehangWu_v7.csv', 
                        # 'Testing_data3/Mid/Hao/data_0_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_1_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_2_Hao6_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_4_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_5_Hao6_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_7_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_8_Hao6_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_Hao6_v7.csv', 
                        # 'Testing_data3/Mid/Hao/data_0_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_1_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_2_Hao7_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_4_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_5_Hao7_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_7_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_8_Hao7_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_Hao7_v7.csv',                         
                        # 'Testing_data3/Mid/Hao/data_0_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_1_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_2_Liangting_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_4_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_5_Liangting_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_7_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_8_Liangting_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_Liangting_v7.csv', #26%
                        # 'Testing_data3/Mid/Hao/data_0_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_1_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_2_ShuxinChen_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_4_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_5_ShuxinChen_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_7_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_8_ShuxinChen_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_ShuxinChen_v7.csv', #17.5
                        # 'Testing_data3/Mid/Hao/data_0_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_1_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_2_TeacherWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_4_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_5_TeacherWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_7_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_8_TeacherWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_TeacherWu_v7.csv',  #13.1
                        # 'Testing_data3/Mid/Hao/data_0_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_1_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_2_Yay_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_4_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_5_Yay_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_7_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_8_Yay_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_Yay_v7.csv', #13.1
                        # 'Testing_data3/Mid/Hao/data_0_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_1_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_2_ShixunWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_4_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_5_ShixunWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_7_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_8_ShixunWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_ShixunWu_v7.csv', #34                        

                        ]
label_test_filenames_new = [
'Testing_data/HaozheTang/data_0_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_1_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_2_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_3_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_4_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_5_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_6_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_7_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_8_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_9_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/Qing/data_0_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_1_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_2_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_3_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_4_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_5_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_6_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_7_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_8_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_9_Qing_v7.csv_label_v1.txt',
'Testing_data/Serge/data_0_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_1_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_2_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_3_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_4_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_5_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_6_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_7_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_8_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_9_Serge_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_0_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_1_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_2_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_3_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_4_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_5_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_6_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_7_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_8_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_9_AlexCui_v7.csv_label_v1.txt',  
'Testing_data/Keyarash/data_0_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_1_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_2_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_3_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_4_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_5_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_6_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_7_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_8_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_9_Keyarash_v7.csv_label_v1.txt', 
'Testing_data/PingweiJiao/data_0_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_1_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_2_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_4_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_5_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_7_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_8_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_9_PingweiJiao_v7.csv_label_v1.txt',  
'Testing_data/Talia/data_1_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_2_Talia_v7.csv_label_v1.txt',
'Testing_data/Talia/data_3_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_4_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_5_Talia_v7.csv_label_v1.txt',
'Testing_data/Talia/data_7_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_9_Talia_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_0_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_1_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_2_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_3_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_4_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_5_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_6_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_7_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_8_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_9_Vineet_v7.csv_label_v1.txt',      
'Testing_data/Song/data_0_Song_v7.csv_label_v1.txt','Testing_data/Song/data_1_Song_v7.csv_label_v1.txt','Testing_data/Song/data_2_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_3_Song_v7.csv_label_v1.txt','Testing_data/Song/data_4_Song_v7.csv_label_v1.txt','Testing_data/Song/data_5_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_6_Song_v7.csv_label_v1.txt','Testing_data/Song/data_7_Song_v7.csv_label_v1.txt','Testing_data/Song/data_8_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_9_Song_v7.csv_label_v1.txt',   
'Testing_data/Vito/data_0_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_1_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_2_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_3_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_4_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_5_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_6_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_7_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_8_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_9_Vito_v7.csv_label_v1.txt',  
'Testing_data/YangLi/data_0_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_1_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_2_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_3_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_4_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_5_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_6_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_7_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_9_YangLi_v7.csv_label_v1.txt',  
'Testing_data/HanTing/data_0_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_1_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_2_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_3_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_4_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_5_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_6_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_7_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_8_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_9_HanTing_v7.csv_label_v1.txt',  
'Testing_data/Naram/data_1_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_2_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_3_Naram_v7.csv_label_v1.txt',
'Testing_data/Naram/data_4_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_5_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_6_Naram_v7.csv_label_v1.txt',
'Testing_data/Naram/data_7_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_8_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_9_Naram_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_0_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_1_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_2_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_3_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_4_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_5_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_6_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_7_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_8_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_9_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_0_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_1_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_2_Weichuan_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_3_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_4_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_5_Weichuan_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_6_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_7_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_8_Weichuan_v7.csv_label_v1.txt',               
'Testing_data/jundaWu/data_0_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_1_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_2_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_3_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_4_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_5_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_6_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_7_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_8_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_9_jundaWu_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_0_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_1_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_2_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_3_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_4_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_5_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_6_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_7_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_8_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_9_shiXun_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_0_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_1_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_2_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_3_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_4_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_5_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_6_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_7_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_8_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_9_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_0_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_3_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_6_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_9_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/liang/data_0_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_1_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_2_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_3_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_4_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_5_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_6_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_7_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_8_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_9_Liang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_0_TingLiang_v7.csv_label_v1.txt', 'Testing_data/TingLiang/data_1_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_2_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_3_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_4_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_5_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_6_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_7_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_8_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_9_TingLiang_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_ZehangWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_ZehangWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_ZehangWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_ZehangWu_v7.csv_label_v1.txt',  
                        # 'Testing_data3/Mid/Hao/data_0_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_Hao6_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_Hao6_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_Hao6_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_Hao6_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_Hao7_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_Hao7_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_Hao7_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_Hao7_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_Liangting_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_Liangting_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_Liangting_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_Liangting_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_ShuxinChen_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_ShuxinChen_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_ShuxinChen_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_ShuxinChen_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_TeacherWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_TeacherWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_TeacherWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_TeacherWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_Yay_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_Yay_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_Yay_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_Yay_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_ShixunWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_ShixunWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_ShixunWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_ShixunWu_v7.csv_label_v1.txt',                                                                                                                                                                                                              
                        ]

### 3.13 load data

In [ ]:
def create_label_frame(file_name):
  label_frame = pd.DataFrame(columns = ['label'])
  for i, file in enumerate(file_name):
    data = pd.read_csv(file, names = ['label'])
    label_frame = pd.concat([label_frame, data], axis = 0, ignore_index = True)
  return label_frame

def create_data_frame(file_name):
  data_frame = []
  for i, file in enumerate(file_name):
    data = pd.read_csv(file)
    data_frame.append (data)
  data_frame = pd.concat(data_frame, join = 'outer')
  return data_frame

def data_3D_transfer(photodiode_arr, data_frame):
  num_channels = len(photodiode_arr)
  data_3D = list()
  for i, photodiode_name in enumerate(photodiode_arr):
    data_3D.append(data_frame[photodiode_name].values.reshape(-1, ))  
  data_3D = dstack(data_3D)
  data_3D = data_3D.reshape((-1, 300, num_channels))
  return data_3D

def load_dataset(photodiode_arr):
                                         
  label_frame = create_label_frame(label_filenames)
  label_test_frame_new = create_label_frame(label_test_filenames_new)  
  data_frame = create_data_frame(data_filenames)
  data_test_frame_new = create_data_frame(data_test_filenames_new)  
  data_3D = data_3D_transfer(photodiode_arr, data_frame)
  data_test_3D_new = data_3D_transfer(photodiode_arr, data_test_frame_new)  
  data_y = label_frame
  data_test_y_new = label_test_frame_new  
  print('data_y : ', len(data_y))
  print('data_test_y : ', len(data_test_y_new))   
  data_y = to_categorical(data_y,num_classes=10) 
  data_test_y_new = to_categorical(data_test_y_new,num_classes=10)  
  Xtra,Xval,Ytra,Yval = train_test_split(data_3D,data_y,test_size=0.2,shuffle=True)
  return  Xtra,Ytra, Xval,Yval,  data_test_3D_new, data_test_y_new

## 3.2 Within-Subjects

### 3.21 training_data

In [ ]:
data_filenames = [
          'Training_data/0/data_0_22_v7.csv', 'Training_data/0/data_0_22_2_v7.csv', 'Training_data/0/data_0_121_s1_t1_1_v7.csv', 'Training_data/0/data_0_211_s1_t1_1_v7.csv', 'Training_data/0/data_0_220_s1_t1_1_v7.csv', 'Training_data/0/data_0_221_s1_t1_1_v7.csv',
          'Training_data/0/data_0_221_s1_t2_1_v7.csv', 'Training_data/0/data_0_221_s1_t3_1_v7.csv', 'Training_data/0/data_0_221_s1_t4_3_v7.csv', 'Training_data/0/data_0_221_s1_t6_3_v7.csv', 'Training_data/0/data_0_221_s2_t1_1_v7.csv', 'Training_data/0/data_0_221_s3_t1_1_v7.csv',
          'Training_data/0/data_0_221_s3_t4_1_v7.csv', 'Training_data/0/data_0_221_s3_t5_1_v7.csv', 'Training_data/0/data_0_221_s4_t1_1_v7.csv','Training_data/0/data_0_221_s5_t1_1_v7.csv', 'Training_data/0/data_0_222_s1_t1_1_v7.csv', 'Training_data/0/data_0_222_s1_t2_1_v7.csv', 
          'Training_data/0/data_0_221_s2_t1_1_v7.csv',  

          'Training_data/1/data_1_011_021_v7.csv', 'Training_data/1/data_1_120_122_v7.csv', 'Training_data/1/data_1_021_s1_t1_1_v7.csv', 'Training_data/1/data_1_120_s1_t1_1_v7.csv','Training_data/1/data_1_122_s1_t1_1_v7.csv', 'Training_data/1/data_1_021_s2_t1_1_v7.csv',
          'Training_data/1/data_1_221_s3_t1_1_v7.csv', 'Training_data/1/data_1_221_s4_t1_1_v7.csv', 'Training_data/1/data_1_221_s5_t1_1_v7.csv', 'Training_data/1/data_1_221_s6_t1_1_v7.csv', 'Training_data/1/data_1_221_s3_t1_2_v7.csv', 'Training_data/1/data_1_221_s4_t1_3_v7.csv',
          'Training_data/1/data_1_222_s1_t1_1_v7.csv', 'Training_data/1/data_1_222_s1_t2_1_v7.csv', 'Training_data/1/data_1_222_s2_t1_1_v7.csv',

          'Training_data/2/data_2_22_v7.csv', 'Training_data/2/data_2_22_2_v7.csv', 'Training_data/2/data_2_121_s1_t1_1_v7.csv', 'Training_data/2/data_2_211_s1_t1_1_v7.csv', 'Training_data/2/data_2_220_s1_t1_1_v7.csv','Training_data/2/data_2_221_s1_t1_1_v7.csv',
          'Training_data/2/data_2_222_s1_t1_1_v7.csv', 'Training_data/2/data_2_221_s2_t1_1_v7.csv', 'Training_data/2/data_2_221_s3_t1_1_v7.csv', 'Training_data/2/data_2_221_s3_t1_3_v7.csv','Training_data/2/data_2_222_s2_t1_1_v7.csv', 'Training_data/2/data_2_222_s2_t2_1_v7.csv',
          'Training_data/2/data_2_222_s3_t1_1_v7.csv',

          'Training_data/3/data_3_22_v7.csv','Training_data/3/data_3_22_2_v7.csv','Training_data/3/data_3_121_s1_t1_1_v7.csv','Training_data/3/data_3_121_s2_t1_1_v7.csv', 'Training_data/3/data_3_220_s1_t1_1_v7.csv','Training_data/3/data_3_220_s2_t1_1_v7.csv',
          'Training_data/3/data_3_221_s1_t1_1_v7.csv','Training_data/3/data_3_221_s2_t1_1_v7.csv', 'Training_data/3/data_3_222_s1_t1_1_v7.csv','Training_data/3/data_3_222_s1_t2_1_v7.csv', 'Training_data/3/data_3_222_s2_t1_1_v7.csv','Training_data/3/data_3_222_s2_t2_1_v7.csv',  
                  
          'Training_data/4/data_4_221_v7.csv', 'Training_data/4/data_4_221_2_v7.csv','Training_data/4/data_4_121_s1_t1_1_v7.csv','Training_data/4/data_4_121_s2_t1_1_v7.csv', 'Training_data/4/data_4_211_s1_t1_1_v7.csv', 'Training_data/4/data_4_211_s2_t1_1_v7.csv', 
          'Training_data/4/data_4_210_s1_t1_1_v7.csv','Training_data/4/data_4_210_s2_t1_1_v7.csv','Training_data/4/data_4_220_s1_t1_1_v7.csv','Training_data/4/data_4_220_s2_t1_1_v7.csv','Training_data/4/data_4_221_s1_t1_1_v7.csv','Training_data/4/data_4_221_s2_t1_1_v7.csv',
          'Training_data/4/data_4_221_s2_t2_1_v7.csv','Training_data/4/data_4_221_s2_t1_3_v7.csv','Training_data/4/data_4_221_s2_t2_2_v7.csv','Training_data/4/data_4_221_s3_t1_1_v7.csv','Training_data/4/data_4_221_s4_t1_1_v7.csv','Training_data/4/data_4_221_s5_t1_1_v7.csv',
          'Training_data/4/data_4_221_s6_t1_1_v7.csv','Training_data/4/data_4_221_s6_t1_2_v7.csv','Training_data/4/data_4_221_s6_t1_3_v7.csv','Training_data/4/data_4_222_s1_t1_1_v7.csv','Training_data/4/data_4_222_s1_t2_1_v7.csv','Training_data/4/data_4_222_s2_t1_1_v7.csv',

          'Training_data/5/data_5_221_v7.csv', 'Training_data/5/data_5_221_2_v7.csv', 'Training_data/5/data_5_121_s1_t1_1_v7.csv','Training_data/5/data_5_121_s1_t2_1_v7.csv','Training_data/5/data_5_220_s1_t1_1_v7.csv','Training_data/5/data_5_220_s1_t2_1_v7.csv',
          'Training_data/5/data_5_221_s1_t1_1_v7.csv','Training_data/5/data_5_221_s1_t2_1_v7.csv','Training_data/5/data_5_221_s1_t3_1_v7.csv','Training_data/5/data_5_221_s1_t3_3_v7.csv','Training_data/5/data_5_221_s2_t2_3_v7.csv','Training_data/5/data_5_221_s3_t2_1_v7.csv',
          'Training_data/5/data_5_221_s3_t3_1_v7.csv','Training_data/5/data_5_221_s3_t3_2_v7.csv','Training_data/5/data_5_222_s1_t1_1_v7.csv','Training_data/5/data_5_222_s1_t2_1_v7.csv','Training_data/5/data_5_222_s2_t1_1_v7.csv','Training_data/5/data_5_222_s2_t2_1_v7.csv',

          'Training_data/6/data_6_121_v7.csv', 'Training_data/6/data_6_221_220_v7.csv', 'Training_data/6/data_6_121_s1_t1_1_v7.csv','Training_data/6/data_6_221_s1_t1_1_v7.csv','Training_data/6/data_6_221_s2_t1_1_v7.csv','Training_data/6/data_6_221_s3_t1_1_v7.csv',
          'Training_data/6/data_6_221_s3_t1_3_v7.csv','Training_data/6/data_6_221_s3_t2_1_v7.csv','Training_data/6/data_6_221_s3_t2_2_v7.csv','Training_data/6/data_6_221_s3_t2_3_v7.csv','Training_data/6/data_6_222_s1_t1_1_v7.csv','Training_data/6/data_6_222_s1_t2_1_v7.csv',
          'Training_data/6/data_6_222_s2_t1_1_v7.csv','Training_data/6/data_6_222_s2_t2_1_v7.csv',
                  
          'Training_data/7/data_7_221_v7.csv','Training_data/7/data_7_221_3_v7.csv', 'Training_data/7/data_7_221_2_v7.csv', 'Training_data/7/data_7_121_s1_t1_1_v7.csv','Training_data/7/data_7_121_s2_t1_1_v7.csv','Training_data/7/data_7_121_s3_t1_1_v7.csv', 
          'Training_data/7/data_7_221_s1_t1_1_v7.csv','Training_data/7/data_7_221_s2_t1_1_v7.csv','Training_data/7/data_7_221_s3_t1_1_v7.csv','Training_data/7/data_7_221_s4_t1_1_v7.csv','Training_data/7/data_7_221_s5_t1_1_v7.csv','Training_data/7/data_7_221_s5_t1_3_v7.csv',
          'Training_data/7/data_7_221_s6_t1_1_v7.csv','Training_data/7/data_7_221_s7_t1_1_v7.csv','Training_data/7/data_7_221_s7_t1_3_v7.csv','Training_data/7/data_7_222_s1_t1_1_v7.csv','Training_data/7/data_7_222_s1_t2_1_v7.csv','Training_data/7/data_7_222_s2_t1_1_v7.csv',
          'Training_data/7/data_7_222_s2_t2_1_v7.csv',   
                  
          'Training_data/8/data_8_221_v7.csv', 'Training_data/8/data_8_221_3_v7.csv', 'Training_data/8/data_8_121_s1_t1_1_v7.csv','Training_data/8/data_8_121_s1_t2_1_v7.csv','Training_data/8/data_8_121_s1_t3_1_v7.csv', 'Training_data/8/data_8_220_s1_t1_1_v7.csv',
          'Training_data/8/data_8_220_s1_t2_1_v7.csv','Training_data/8/data_8_220_s1_t3_1_v7.csv','Training_data/8/data_8_221_s1_t1_1_v7.csv','Training_data/8/data_8_221_s1_t2_1_v7.csv','Training_data/8/data_8_221_s1_t3_1_v7.csv','Training_data/8/data_8_221_s1_t4_1_v7.csv',
          'Training_data/8/data_8_221_s1_t5_1_v7.csv','Training_data/8/data_8_121_s1_t4_1_v7.csv','Training_data/8/data_8_221_s1_t6_1_v7.csv','Training_data/8/data_8_221_s1_t7_1_v7.csv','Training_data/8/data_8_221_s1_t7_2_v7.csv','Training_data/8/data_8_221_s1_t7_3_v7.csv',
          'Training_data/8/data_8_221_s1_t8_1_v7.csv','Training_data/8/data_8_221_s1_t8_3_v7.csv','Training_data/8/data_8_221_s2_t1_1_v7.csv','Training_data/8/data_8_222_s1_t1_1_v7.csv','Training_data/8/data_8_222_s1_t2_1_v7.csv','Training_data/8/data_8_222_s2_t1_1_v7.csv',
                  
          'Training_data/9/data_9_120_v7.csv', 'Training_data/9/data_9_221_220_v7.csv','Training_data/9/data_9_221_220_3_v7.csv','Training_data/9/data_9_120_s1_t1_1_v7.csv','Training_data/9/data_9_121_s1_t1_1_v7.csv','Training_data/9/data_9_220_s1_t1_1_v7.csv',
          'Training_data/9/data_9_221_s1_t1_1_v7.csv','Training_data/9/data_9_220_s1_t2_1_v7.csv','Training_data/9/data_9_221_s1_t2_2_v7.csv','Training_data/9/data_9_221_s1_t2_3_v7.csv','Training_data/9/data_9_221_s2_t1_1_v7.csv','Training_data/9/data_9_221_s3_t1_1_v7.csv',
          'Training_data/9/data_9_221_s3_t1_2_v7.csv' ,'Training_data/9/data_9_221_s3_t1_3_v7.csv' ,'Training_data/9/data_9_221_s4_t1_3_v7.csv','Training_data/9/data_9_222_s1_t1_1_v7.csv','Training_data/9/data_9_222_s1_t2_1_v7.csv','Training_data/9/data_9_222_s2_t1_1_v7.csv',


'Testing_data/HaozheTang/data_0_HaozheTang_v7.csv','Testing_data/HaozheTang/data_1_HaozheTang_v7.csv','Testing_data/HaozheTang/data_2_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_3_HaozheTang_v7.csv','Testing_data/HaozheTang/data_4_HaozheTang_v7.csv','Testing_data/HaozheTang/data_5_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_6_HaozheTang_v7.csv','Testing_data/HaozheTang/data_7_HaozheTang_v7.csv','Testing_data/HaozheTang/data_8_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_9_HaozheTang_v7.csv',
'Testing_data/Qing/data_0_Qing_v7.csv', 'Testing_data/Qing/data_1_Qing_v7.csv', 'Testing_data/Qing/data_2_Qing_v7.csv',
'Testing_data/Qing/data_3_Qing_v7.csv', 'Testing_data/Qing/data_4_Qing_v7.csv', 'Testing_data/Qing/data_5_Qing_v7.csv',
'Testing_data/Qing/data_6_Qing_v7.csv', 'Testing_data/Qing/data_7_Qing_v7.csv', 'Testing_data/Qing/data_8_Qing_v7.csv',
'Testing_data/Qing/data_9_Qing_v7.csv',
'Testing_data/Serge/data_0_Serge_v7.csv','Testing_data/Serge/data_1_Serge_v7.csv','Testing_data/Serge/data_2_Serge_v7.csv',
'Testing_data/Serge/data_3_Serge_v7.csv','Testing_data/Serge/data_4_Serge_v7.csv','Testing_data/Serge/data_5_Serge_v7.csv',
'Testing_data/Serge/data_6_Serge_v7.csv','Testing_data/Serge/data_7_Serge_v7.csv','Testing_data/Serge/data_8_Serge_v7.csv',
'Testing_data/Serge/data_9_Serge_v7.csv',
'Testing_data/AlexCui/data_0_AlexCui_v7.csv','Testing_data/AlexCui/data_1_AlexCui_v7.csv','Testing_data/AlexCui/data_2_AlexCui_v7.csv',
'Testing_data/AlexCui/data_3_AlexCui_v7.csv','Testing_data/AlexCui/data_4_AlexCui_v7.csv','Testing_data/AlexCui/data_5_AlexCui_v7.csv',
'Testing_data/AlexCui/data_6_AlexCui_v7.csv','Testing_data/AlexCui/data_7_AlexCui_v7.csv','Testing_data/AlexCui/data_8_AlexCui_v7.csv',
'Testing_data/AlexCui/data_9_AlexCui_v7.csv', 
'Testing_data/Keyarash/data_0_Keyarash_v7.csv','Testing_data/Keyarash/data_1_Keyarash_v7.csv','Testing_data/Keyarash/data_2_Keyarash_v7.csv',
'Testing_data/Keyarash/data_3_Keyarash_v7.csv','Testing_data/Keyarash/data_4_Keyarash_v7.csv','Testing_data/Keyarash/data_5_Keyarash_v7.csv',
'Testing_data/Keyarash/data_6_Keyarash_v7.csv','Testing_data/Keyarash/data_7_Keyarash_v7.csv','Testing_data/Keyarash/data_8_Keyarash_v7.csv',
'Testing_data/Keyarash/data_9_Keyarash_v7.csv',
'Testing_data/PingweiJiao/data_0_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_1_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_2_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_4_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_5_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_7_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_8_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_9_PingweiJiao_v7.csv',                        
'Testing_data/Talia/data_1_Talia_v7.csv','Testing_data/Talia/data_2_Talia_v7.csv',
'Testing_data/Talia/data_3_Talia_v7.csv','Testing_data/Talia/data_4_Talia_v7.csv','Testing_data/Talia/data_5_Talia_v7.csv',
'Testing_data/Talia/data_7_Talia_v7.csv',
'Testing_data/Talia/data_9_Talia_v7.csv',
'Testing_data/Vineet/data_0_Vineet_v7.csv','Testing_data/Vineet/data_1_Vineet_v7.csv','Testing_data/Vineet/data_2_Vineet_v7.csv',
'Testing_data/Vineet/data_3_Vineet_v7.csv','Testing_data/Vineet/data_4_Vineet_v7.csv','Testing_data/Vineet/data_5_Vineet_v7.csv',
'Testing_data/Vineet/data_6_Vineet_v7.csv','Testing_data/Vineet/data_7_Vineet_v7.csv','Testing_data/Vineet/data_8_Vineet_v7.csv',
'Testing_data/Vineet/data_9_Vineet_v7.csv',   
'Testing_data/Song/data_0_Song_v7.csv','Testing_data/Song/data_1_Song_v7.csv','Testing_data/Song/data_2_Song_v7.csv',
'Testing_data/Song/data_3_Song_v7.csv','Testing_data/Song/data_4_Song_v7.csv','Testing_data/Song/data_5_Song_v7.csv',
'Testing_data/Song/data_6_Song_v7.csv','Testing_data/Song/data_7_Song_v7.csv','Testing_data/Song/data_8_Song_v7.csv',
'Testing_data/Song/data_9_Song_v7.csv',   
'Testing_data/Vito/data_0_Vito_v7.csv','Testing_data/Vito/data_1_Vito_v7.csv','Testing_data/Vito/data_2_Vito_v7.csv',
'Testing_data/Vito/data_3_Vito_v7.csv','Testing_data/Vito/data_4_Vito_v7.csv','Testing_data/Vito/data_5_Vito_v7.csv',
'Testing_data/Vito/data_6_Vito_v7.csv','Testing_data/Vito/data_7_Vito_v7.csv','Testing_data/Vito/data_8_Vito_v7.csv',
'Testing_data/Vito/data_9_Vito_v7.csv', 
'Testing_data/YangLi/data_0_YangLi_v7.csv','Testing_data/YangLi/data_1_YangLi_v7.csv','Testing_data/YangLi/data_2_YangLi_v7.csv',
'Testing_data/YangLi/data_3_YangLi_v7.csv','Testing_data/YangLi/data_4_YangLi_v7.csv','Testing_data/YangLi/data_5_YangLi_v7.csv',
'Testing_data/YangLi/data_6_YangLi_v7.csv','Testing_data/YangLi/data_7_YangLi_v7.csv',
'Testing_data/YangLi/data_9_YangLi_v7.csv',
'Testing_data/HanTing/data_0_HanTing_v7.csv','Testing_data/HanTing/data_1_HanTing_v7.csv','Testing_data/HanTing/data_2_HanTing_v7.csv',
'Testing_data/HanTing/data_3_HanTing_v7.csv','Testing_data/HanTing/data_4_HanTing_v7.csv','Testing_data/HanTing/data_5_HanTing_v7.csv',
'Testing_data/HanTing/data_6_HanTing_v7.csv','Testing_data/HanTing/data_7_HanTing_v7.csv','Testing_data/HanTing/data_8_HanTing_v7.csv',
'Testing_data/HanTing/data_9_HanTing_v7.csv',
'Testing_data/Naram/data_1_Naram_v7.csv','Testing_data/Naram/data_2_Naram_v7.csv','Testing_data/Naram/data_3_Naram_v7.csv',
'Testing_data/Naram/data_4_Naram_v7.csv','Testing_data/Naram/data_5_Naram_v7.csv','Testing_data/Naram/data_6_Naram_v7.csv',
'Testing_data/Naram/data_7_Naram_v7.csv','Testing_data/Naram/data_8_Naram_v7.csv','Testing_data/Naram/data_9_Naram_v7.csv',   
'Testing_data/Ricardo/data_0_Ricardo_v7.csv','Testing_data/Ricardo/data_1_Ricardo_v7.csv','Testing_data/Ricardo/data_2_Ricardo_v7.csv',
'Testing_data/Ricardo/data_3_Ricardo_v7.csv','Testing_data/Ricardo/data_4_Ricardo_v7.csv','Testing_data/Ricardo/data_5_Ricardo_v7.csv',
'Testing_data/Ricardo/data_6_Ricardo_v7.csv','Testing_data/Ricardo/data_7_Ricardo_v7.csv','Testing_data/Ricardo/data_8_Ricardo_v7.csv',
'Testing_data/Ricardo/data_9_Ricardo_v7.csv',
'Testing_data/Weichuan/data_0_Weichuan_v7.csv','Testing_data/Weichuan/data_1_Weichuan_v7.csv','Testing_data/Weichuan/data_2_Weichuan_v7.csv',
'Testing_data/Weichuan/data_3_Weichuan_v7.csv','Testing_data/Weichuan/data_4_Weichuan_v7.csv','Testing_data/Weichuan/data_5_Weichuan_v7.csv',
'Testing_data/Weichuan/data_6_Weichuan_v7.csv','Testing_data/Weichuan/data_7_Weichuan_v7.csv','Testing_data/Weichuan/data_8_Weichuan_v7.csv',       
'Testing_data/jundaWu/data_0_jundaWu_v7.csv','Testing_data/jundaWu/data_1_jundaWu_v7.csv','Testing_data/jundaWu/data_2_jundaWu_v7.csv',
'Testing_data/jundaWu/data_3_jundaWu_v7.csv','Testing_data/jundaWu/data_4_jundaWu_v7.csv','Testing_data/jundaWu/data_5_jundaWu_v7.csv',
'Testing_data/jundaWu/data_6_jundaWu_v7.csv','Testing_data/jundaWu/data_7_jundaWu_v7.csv','Testing_data/jundaWu/data_8_jundaWu_v7.csv',
'Testing_data/jundaWu/data_9_jundaWu_v7.csv',
'Testing_data/shixunWu/data_0_shiXun_v7.csv','Testing_data/shixunWu/data_1_shiXun_v7.csv','Testing_data/shixunWu/data_2_shiXun_v7.csv',
'Testing_data/shixunWu/data_3_shiXun_v7.csv','Testing_data/shixunWu/data_4_shiXun_v7.csv','Testing_data/shixunWu/data_5_shiXun_v7.csv',
'Testing_data/shixunWu/data_6_shiXun_v7.csv','Testing_data/shixunWu/data_7_shiXun_v7.csv','Testing_data/shixunWu/data_8_shiXun_v7.csv',
'Testing_data/shixunWu/data_9_shiXun_v7.csv',
'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv',
'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv',
'Testing_data/shuxinChen/data_0_shuxinChen_v7.csv','Testing_data/shuxinChen/data_1_shuxinChen_v7.csv','Testing_data/shuxinChen/data_2_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_3_shuxinChen_v7.csv','Testing_data/shuxinChen/data_4_shuxinChen_v7.csv','Testing_data/shuxinChen/data_5_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_6_shuxinChen_v7.csv','Testing_data/shuxinChen/data_7_shuxinChen_v7.csv','Testing_data/shuxinChen/data_8_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_9_shuxinChen_v7.csv',
'Testing_data/zehangWu/data_0_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_3_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_6_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_9_ZehangWu_v7.csv',
'Testing_data/liang/data_0_Liang_v7.csv','Testing_data/liang/data_1_Liang_v7.csv','Testing_data/liang/data_2_Liang_v7.csv',
'Testing_data/liang/data_3_Liang_v7.csv','Testing_data/liang/data_4_Liang_v7.csv','Testing_data/liang/data_5_Liang_v7.csv',
'Testing_data/liang/data_6_Liang_v7.csv','Testing_data/liang/data_7_Liang_v7.csv','Testing_data/liang/data_8_Liang_v7.csv',
'Testing_data/liang/data_9_Liang_v7.csv',
'Testing_data/TingLiang/data_0_TingLiang_v7.csv', 'Testing_data/TingLiang/data_1_TingLiang_v7.csv','Testing_data/TingLiang/data_2_TingLiang_v7.csv',
'Testing_data/TingLiang/data_3_TingLiang_v7.csv','Testing_data/TingLiang/data_4_TingLiang_v7.csv','Testing_data/TingLiang/data_5_TingLiang_v7.csv',
'Testing_data/TingLiang/data_6_TingLiang_v7.csv','Testing_data/TingLiang/data_7_TingLiang_v7.csv','Testing_data/TingLiang/data_8_TingLiang_v7.csv',
'Testing_data/TingLiang/data_9_TingLiang_v7.csv',      
          ]


label_filenames = [
'Training_data/0/data_0_22_v7.csv_label_v1.txt','Training_data/0/data_0_22_2_v7.csv_label_v1.txt','Training_data/0/data_0_121_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_211_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_220_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t1_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/0/data_0_221_s1_t3_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t4_3_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t6_3_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s2_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s3_t1_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s3_t4_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s3_t5_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/0/data_0_221_s5_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_222_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_222_s1_t2_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s2_t1_1_v7.csv_label_v1.txt', 

'Training_data/1/data_1_011_021_v7.csv_label_v1.txt','Training_data/1/data_1_120_122_v7.csv_label_v1.txt', 'Training_data/1/data_1_021_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_120_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_122_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_021_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/1/data_1_221_s3_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s5_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s6_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s3_t1_2_v7.csv_label_v1.txt','Training_data/1/data_1_221_s4_t1_3_v7.csv_label_v1.txt',
'Training_data/1/data_1_222_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/1/data_1_222_s1_t2_1_v7.csv_label_v1.txt', 'Training_data/1/data_1_222_s2_t1_1_v7.csv_label_v1.txt',

'Training_data/2/data_2_22_v7.csv_label_v1.txt','Training_data/2/data_2_22_2_v7.csv_label_v1.txt','Training_data/2/data_2_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_211_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/2/data_2_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s3_t1_1_v7.csv_label_v1.txt', 'Training_data/2/data_2_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/2/data_2_222_s2_t1_1_v7.csv_label_v1.txt', 'Training_data/2/data_2_222_s2_t2_1_v7.csv_label_v1.txt',
'Training_data/2/data_2_222_s3_t1_1_v7.csv_label_v1.txt', 
                  
'Training_data/3/data_3_22_v7.csv_label_v1.txt','Training_data/3/data_3_22_2_v7.csv_label_v1.txt','Training_data/3/data_3_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_220_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/3/data_3_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s1_t2_1_v7.csv_label_v1.txt', 'Training_data/3/data_3_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s2_t2_1_v7.csv_label_v1.txt',  
                  
'Training_data/4/data_4_221_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_2_v7.csv_label_v1.txt','Training_data/4/data_4_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_211_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/4/data_4_211_s2_t1_1_v7.csv_label_v1.txt', 
'Training_data/4/data_4_210_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_210_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_220_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/4/data_4_221_s2_t2_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t1_3_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t2_2_v7.csv_label_v1.txt','Training_data/4/data_4_221_s3_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s5_t1_1_v7.csv_label_v1.txt',
'Training_data/4/data_4_221_s6_t1_1_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_s6_t1_2_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_s6_t1_3_v7.csv_label_v1.txt','Training_data/4/data_4_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/4/data_4_222_s2_t1_1_v7.csv_label_v1.txt', 
                  
'Training_data/5/data_5_221_v7.csv_label_v1.txt', 'Training_data/5/data_5_221_2_v7.csv_label_v1.txt', 'Training_data/5/data_5_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_121_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_220_s1_t2_1_v7.csv_label_v1.txt',
'Training_data/5/data_5_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t3_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t3_3_v7.csv_label_v1.txt','Training_data/5/data_5_221_s2_t2_3_v7.csv_label_v1.txt','Training_data/5/data_5_221_s3_t2_1_v7.csv_label_v1.txt',
'Training_data/5/data_5_221_s3_t3_1_v7.csv_label_v1.txt', 'Training_data/5/data_5_221_s3_t3_2_v7.csv_label_v1.txt','Training_data/5/data_5_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s2_t2_1_v7.csv_label_v1.txt', 
                  
'Training_data/6/data_6_121_v7.csv_label_v1.txt', 'Training_data/6/data_6_221_220_v7.csv_label_v1.txt', 'Training_data/6/data_6_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t1_1_v7.csv_label_v1.txt',
'Training_data/6/data_6_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_2_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_3_v7.csv_label_v1.txt','Training_data/6/data_6_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_222_s1_t2_1_v7.csv_label_v1.txt',
'Training_data/6/data_6_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_222_s2_t2_1_v7.csv_label_v1.txt',
                  
'Training_data/7/data_7_221_v7.csv_label_v1.txt','Training_data/7/data_7_221_3_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_2_v7.csv_label_v1.txt', 'Training_data/7/data_7_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_121_s3_t1_1_v7.csv_label_v1.txt', 
'Training_data/7/data_7_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_221_s3_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s4_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s5_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s5_t1_3_v7.csv_label_v1.txt', 
'Training_data/7/data_7_221_s6_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s7_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s7_t1_3_v7.csv_label_v1.txt','Training_data/7/data_7_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/7/data_7_222_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/7/data_7_222_s2_t2_1_v7.csv_label_v1.txt',   
                  
'Training_data/8/data_8_221_v7.csv_label_v1.txt', 'Training_data/8/data_8_221_3_v7.csv_label_v1.txt', 'Training_data/8/data_8_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t3_1_v7.csv_label_v1.txt', 'Training_data/8/data_8_220_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/8/data_8_220_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_220_s1_t3_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t3_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t4_1_v7.csv_label_v1.txt',
'Training_data/8/data_8_221_s1_t5_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t4_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t6_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_2_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_3_v7.csv_label_v1.txt',
'Training_data/8/data_8_221_s1_t8_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t8_3_v7.csv_label_v1.txt','Training_data/8/data_8_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s2_t1_1_v7.csv_label_v1.txt',
                  
'Training_data/9/data_9_120_v7.csv_label_v1.txt', 'Training_data/9/data_9_221_220_v7.csv_label_v1.txt','Training_data/9/data_9_221_220_3_v7.csv_label_v1.txt','Training_data/9/data_9_120_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_220_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/9/data_9_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_220_s1_t2_1_v7.csv_label_v1.txt','Training_data/9/data_9_221_s1_t2_2_v7.csv_label_v1.txt','Training_data/9/data_9_221_s1_t2_3_v7.csv_label_v1.txt','Training_data/9/data_9_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_221_s3_t1_1_v7.csv_label_v1.txt',
'Training_data/9/data_9_221_s3_t1_2_v7.csv_label_v1.txt','Training_data/9/data_9_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/9/data_9_221_s4_t1_3_v7.csv_label_v1.txt','Training_data/9/data_9_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/9/data_9_222_s2_t1_1_v7.csv_label_v1.txt',
                

'Testing_data/HaozheTang/data_0_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_1_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_2_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_3_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_4_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_5_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_6_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_7_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_8_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_9_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/Qing/data_0_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_1_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_2_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_3_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_4_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_5_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_6_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_7_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_8_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_9_Qing_v7.csv_label_v1.txt',
'Testing_data/Serge/data_0_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_1_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_2_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_3_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_4_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_5_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_6_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_7_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_8_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_9_Serge_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_0_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_1_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_2_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_3_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_4_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_5_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_6_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_7_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_8_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_9_AlexCui_v7.csv_label_v1.txt',  
'Testing_data/Keyarash/data_0_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_1_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_2_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_3_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_4_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_5_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_6_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_7_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_8_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_9_Keyarash_v7.csv_label_v1.txt', 
'Testing_data/PingweiJiao/data_0_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_1_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_2_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_4_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_5_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_7_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_8_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_9_PingweiJiao_v7.csv_label_v1.txt',  
'Testing_data/Talia/data_1_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_2_Talia_v7.csv_label_v1.txt',
'Testing_data/Talia/data_3_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_4_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_5_Talia_v7.csv_label_v1.txt',
'Testing_data/Talia/data_7_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_9_Talia_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_0_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_1_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_2_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_3_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_4_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_5_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_6_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_7_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_8_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_9_Vineet_v7.csv_label_v1.txt',      
'Testing_data/Song/data_0_Song_v7.csv_label_v1.txt','Testing_data/Song/data_1_Song_v7.csv_label_v1.txt','Testing_data/Song/data_2_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_3_Song_v7.csv_label_v1.txt','Testing_data/Song/data_4_Song_v7.csv_label_v1.txt','Testing_data/Song/data_5_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_6_Song_v7.csv_label_v1.txt','Testing_data/Song/data_7_Song_v7.csv_label_v1.txt','Testing_data/Song/data_8_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_9_Song_v7.csv_label_v1.txt',   
'Testing_data/Vito/data_0_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_1_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_2_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_3_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_4_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_5_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_6_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_7_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_8_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_9_Vito_v7.csv_label_v1.txt',  
'Testing_data/YangLi/data_0_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_1_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_2_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_3_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_4_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_5_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_6_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_7_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_9_YangLi_v7.csv_label_v1.txt',  
'Testing_data/HanTing/data_0_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_1_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_2_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_3_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_4_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_5_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_6_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_7_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_8_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_9_HanTing_v7.csv_label_v1.txt',  
'Testing_data/Naram/data_1_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_2_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_3_Naram_v7.csv_label_v1.txt',
'Testing_data/Naram/data_4_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_5_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_6_Naram_v7.csv_label_v1.txt',
'Testing_data/Naram/data_7_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_8_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_9_Naram_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_0_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_1_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_2_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_3_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_4_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_5_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_6_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_7_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_8_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_9_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_0_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_1_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_2_Weichuan_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_3_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_4_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_5_Weichuan_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_6_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_7_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_8_Weichuan_v7.csv_label_v1.txt',               
'Testing_data/jundaWu/data_0_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_1_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_2_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_3_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_4_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_5_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_6_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_7_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_8_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_9_jundaWu_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_0_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_1_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_2_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_3_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_4_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_5_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_6_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_7_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_8_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_9_shiXun_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_0_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_1_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_2_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_3_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_4_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_5_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_6_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_7_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_8_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_9_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_0_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_3_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_6_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_9_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/liang/data_0_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_1_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_2_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_3_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_4_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_5_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_6_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_7_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_8_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_9_Liang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_0_TingLiang_v7.csv_label_v1.txt', 'Testing_data/TingLiang/data_1_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_2_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_3_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_4_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_5_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_6_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_7_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_8_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_9_TingLiang_v7.csv_label_v1.txt',                              
          ]

### 3.22 testing_data

In [ ]:
data_test_filenames_new = [
'Testing_data/HaozheTang/data_0_HaozheTang_v7.csv','Testing_data/HaozheTang/data_1_HaozheTang_v7.csv','Testing_data/HaozheTang/data_2_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_3_HaozheTang_v7.csv','Testing_data/HaozheTang/data_4_HaozheTang_v7.csv','Testing_data/HaozheTang/data_5_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_6_HaozheTang_v7.csv','Testing_data/HaozheTang/data_7_HaozheTang_v7.csv','Testing_data/HaozheTang/data_8_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_9_HaozheTang_v7.csv',
'Testing_data/Qing/data_0_Qing_v7.csv', 'Testing_data/Qing/data_1_Qing_v7.csv', 'Testing_data/Qing/data_2_Qing_v7.csv',
'Testing_data/Qing/data_3_Qing_v7.csv', 'Testing_data/Qing/data_4_Qing_v7.csv', 'Testing_data/Qing/data_5_Qing_v7.csv',
'Testing_data/Qing/data_6_Qing_v7.csv', 'Testing_data/Qing/data_7_Qing_v7.csv', 'Testing_data/Qing/data_8_Qing_v7.csv',
'Testing_data/Qing/data_9_Qing_v7.csv',
'Testing_data/Serge/data_0_Serge_v7.csv','Testing_data/Serge/data_1_Serge_v7.csv','Testing_data/Serge/data_2_Serge_v7.csv',
'Testing_data/Serge/data_3_Serge_v7.csv','Testing_data/Serge/data_4_Serge_v7.csv','Testing_data/Serge/data_5_Serge_v7.csv',
'Testing_data/Serge/data_6_Serge_v7.csv','Testing_data/Serge/data_7_Serge_v7.csv','Testing_data/Serge/data_8_Serge_v7.csv',
'Testing_data/Serge/data_9_Serge_v7.csv',
'Testing_data/AlexCui/data_0_AlexCui_v7.csv','Testing_data/AlexCui/data_1_AlexCui_v7.csv','Testing_data/AlexCui/data_2_AlexCui_v7.csv',
'Testing_data/AlexCui/data_3_AlexCui_v7.csv','Testing_data/AlexCui/data_4_AlexCui_v7.csv','Testing_data/AlexCui/data_5_AlexCui_v7.csv',
'Testing_data/AlexCui/data_6_AlexCui_v7.csv','Testing_data/AlexCui/data_7_AlexCui_v7.csv','Testing_data/AlexCui/data_8_AlexCui_v7.csv',
'Testing_data/AlexCui/data_9_AlexCui_v7.csv', 
'Testing_data/Keyarash/data_0_Keyarash_v7.csv','Testing_data/Keyarash/data_1_Keyarash_v7.csv','Testing_data/Keyarash/data_2_Keyarash_v7.csv',
'Testing_data/Keyarash/data_3_Keyarash_v7.csv','Testing_data/Keyarash/data_4_Keyarash_v7.csv','Testing_data/Keyarash/data_5_Keyarash_v7.csv',
'Testing_data/Keyarash/data_6_Keyarash_v7.csv','Testing_data/Keyarash/data_7_Keyarash_v7.csv','Testing_data/Keyarash/data_8_Keyarash_v7.csv',
'Testing_data/Keyarash/data_9_Keyarash_v7.csv',
'Testing_data/PingweiJiao/data_0_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_1_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_2_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_4_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_5_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_7_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_8_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_9_PingweiJiao_v7.csv',                        
'Testing_data/Talia/data_1_Talia_v7.csv','Testing_data/Talia/data_2_Talia_v7.csv',
'Testing_data/Talia/data_3_Talia_v7.csv','Testing_data/Talia/data_4_Talia_v7.csv','Testing_data/Talia/data_5_Talia_v7.csv',
'Testing_data/Talia/data_7_Talia_v7.csv',
'Testing_data/Talia/data_9_Talia_v7.csv',
'Testing_data/Vineet/data_0_Vineet_v7.csv','Testing_data/Vineet/data_1_Vineet_v7.csv','Testing_data/Vineet/data_2_Vineet_v7.csv',
'Testing_data/Vineet/data_3_Vineet_v7.csv','Testing_data/Vineet/data_4_Vineet_v7.csv','Testing_data/Vineet/data_5_Vineet_v7.csv',
'Testing_data/Vineet/data_6_Vineet_v7.csv','Testing_data/Vineet/data_7_Vineet_v7.csv','Testing_data/Vineet/data_8_Vineet_v7.csv',
'Testing_data/Vineet/data_9_Vineet_v7.csv',   
'Testing_data/Song/data_0_Song_v7.csv','Testing_data/Song/data_1_Song_v7.csv','Testing_data/Song/data_2_Song_v7.csv',
'Testing_data/Song/data_3_Song_v7.csv','Testing_data/Song/data_4_Song_v7.csv','Testing_data/Song/data_5_Song_v7.csv',
'Testing_data/Song/data_6_Song_v7.csv','Testing_data/Song/data_7_Song_v7.csv','Testing_data/Song/data_8_Song_v7.csv',
'Testing_data/Song/data_9_Song_v7.csv',   
'Testing_data/Vito/data_0_Vito_v7.csv','Testing_data/Vito/data_1_Vito_v7.csv','Testing_data/Vito/data_2_Vito_v7.csv',
'Testing_data/Vito/data_3_Vito_v7.csv','Testing_data/Vito/data_4_Vito_v7.csv','Testing_data/Vito/data_5_Vito_v7.csv',
'Testing_data/Vito/data_6_Vito_v7.csv','Testing_data/Vito/data_7_Vito_v7.csv','Testing_data/Vito/data_8_Vito_v7.csv',
'Testing_data/Vito/data_9_Vito_v7.csv', 
'Testing_data/YangLi/data_0_YangLi_v7.csv','Testing_data/YangLi/data_1_YangLi_v7.csv','Testing_data/YangLi/data_2_YangLi_v7.csv',
'Testing_data/YangLi/data_3_YangLi_v7.csv','Testing_data/YangLi/data_4_YangLi_v7.csv','Testing_data/YangLi/data_5_YangLi_v7.csv',
'Testing_data/YangLi/data_6_YangLi_v7.csv','Testing_data/YangLi/data_7_YangLi_v7.csv',
'Testing_data/YangLi/data_9_YangLi_v7.csv',
'Testing_data/HanTing/data_0_HanTing_v7.csv','Testing_data/HanTing/data_1_HanTing_v7.csv','Testing_data/HanTing/data_2_HanTing_v7.csv',
'Testing_data/HanTing/data_3_HanTing_v7.csv','Testing_data/HanTing/data_4_HanTing_v7.csv','Testing_data/HanTing/data_5_HanTing_v7.csv',
'Testing_data/HanTing/data_6_HanTing_v7.csv','Testing_data/HanTing/data_7_HanTing_v7.csv','Testing_data/HanTing/data_8_HanTing_v7.csv',
'Testing_data/HanTing/data_9_HanTing_v7.csv',
'Testing_data/Naram/data_1_Naram_v7.csv','Testing_data/Naram/data_2_Naram_v7.csv','Testing_data/Naram/data_3_Naram_v7.csv',
'Testing_data/Naram/data_4_Naram_v7.csv','Testing_data/Naram/data_5_Naram_v7.csv','Testing_data/Naram/data_6_Naram_v7.csv',
'Testing_data/Naram/data_7_Naram_v7.csv','Testing_data/Naram/data_8_Naram_v7.csv','Testing_data/Naram/data_9_Naram_v7.csv',   
'Testing_data/Ricardo/data_0_Ricardo_v7.csv','Testing_data/Ricardo/data_1_Ricardo_v7.csv','Testing_data/Ricardo/data_2_Ricardo_v7.csv',
'Testing_data/Ricardo/data_3_Ricardo_v7.csv','Testing_data/Ricardo/data_4_Ricardo_v7.csv','Testing_data/Ricardo/data_5_Ricardo_v7.csv',
'Testing_data/Ricardo/data_6_Ricardo_v7.csv','Testing_data/Ricardo/data_7_Ricardo_v7.csv','Testing_data/Ricardo/data_8_Ricardo_v7.csv',
'Testing_data/Ricardo/data_9_Ricardo_v7.csv',
'Testing_data/Weichuan/data_0_Weichuan_v7.csv','Testing_data/Weichuan/data_1_Weichuan_v7.csv','Testing_data/Weichuan/data_2_Weichuan_v7.csv',
'Testing_data/Weichuan/data_3_Weichuan_v7.csv','Testing_data/Weichuan/data_4_Weichuan_v7.csv','Testing_data/Weichuan/data_5_Weichuan_v7.csv',
'Testing_data/Weichuan/data_6_Weichuan_v7.csv','Testing_data/Weichuan/data_7_Weichuan_v7.csv','Testing_data/Weichuan/data_8_Weichuan_v7.csv',       
'Testing_data/jundaWu/data_0_jundaWu_v7.csv','Testing_data/jundaWu/data_1_jundaWu_v7.csv','Testing_data/jundaWu/data_2_jundaWu_v7.csv',
'Testing_data/jundaWu/data_3_jundaWu_v7.csv','Testing_data/jundaWu/data_4_jundaWu_v7.csv','Testing_data/jundaWu/data_5_jundaWu_v7.csv',
'Testing_data/jundaWu/data_6_jundaWu_v7.csv','Testing_data/jundaWu/data_7_jundaWu_v7.csv','Testing_data/jundaWu/data_8_jundaWu_v7.csv',
'Testing_data/jundaWu/data_9_jundaWu_v7.csv',
'Testing_data/shixunWu/data_0_shiXun_v7.csv','Testing_data/shixunWu/data_1_shiXun_v7.csv','Testing_data/shixunWu/data_2_shiXun_v7.csv',
'Testing_data/shixunWu/data_3_shiXun_v7.csv','Testing_data/shixunWu/data_4_shiXun_v7.csv','Testing_data/shixunWu/data_5_shiXun_v7.csv',
'Testing_data/shixunWu/data_6_shiXun_v7.csv','Testing_data/shixunWu/data_7_shiXun_v7.csv','Testing_data/shixunWu/data_8_shiXun_v7.csv',
'Testing_data/shixunWu/data_9_shiXun_v7.csv',
'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv',
'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv',
'Testing_data/shuxinChen/data_0_shuxinChen_v7.csv','Testing_data/shuxinChen/data_1_shuxinChen_v7.csv','Testing_data/shuxinChen/data_2_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_3_shuxinChen_v7.csv','Testing_data/shuxinChen/data_4_shuxinChen_v7.csv','Testing_data/shuxinChen/data_5_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_6_shuxinChen_v7.csv','Testing_data/shuxinChen/data_7_shuxinChen_v7.csv','Testing_data/shuxinChen/data_8_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_9_shuxinChen_v7.csv',
'Testing_data/zehangWu/data_0_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_3_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_6_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_9_ZehangWu_v7.csv',
'Testing_data/liang/data_0_Liang_v7.csv','Testing_data/liang/data_1_Liang_v7.csv','Testing_data/liang/data_2_Liang_v7.csv',
'Testing_data/liang/data_3_Liang_v7.csv','Testing_data/liang/data_4_Liang_v7.csv','Testing_data/liang/data_5_Liang_v7.csv',
'Testing_data/liang/data_6_Liang_v7.csv','Testing_data/liang/data_7_Liang_v7.csv','Testing_data/liang/data_8_Liang_v7.csv',
'Testing_data/liang/data_9_Liang_v7.csv',
'Testing_data/TingLiang/data_0_TingLiang_v7.csv', 'Testing_data/TingLiang/data_1_TingLiang_v7.csv','Testing_data/TingLiang/data_2_TingLiang_v7.csv',
'Testing_data/TingLiang/data_3_TingLiang_v7.csv','Testing_data/TingLiang/data_4_TingLiang_v7.csv','Testing_data/TingLiang/data_5_TingLiang_v7.csv',
'Testing_data/TingLiang/data_6_TingLiang_v7.csv','Testing_data/TingLiang/data_7_TingLiang_v7.csv','Testing_data/TingLiang/data_8_TingLiang_v7.csv',
'Testing_data/TingLiang/data_9_TingLiang_v7.csv',                       

                        ]
label_test_filenames_new = [
'Testing_data/HaozheTang/data_0_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_1_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_2_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_3_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_4_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_5_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_6_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_7_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_8_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_9_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/Qing/data_0_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_1_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_2_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_3_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_4_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_5_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_6_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_7_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_8_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_9_Qing_v7.csv_label_v1.txt',
'Testing_data/Serge/data_0_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_1_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_2_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_3_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_4_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_5_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_6_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_7_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_8_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_9_Serge_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_0_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_1_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_2_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_3_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_4_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_5_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_6_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_7_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_8_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_9_AlexCui_v7.csv_label_v1.txt',  
'Testing_data/Keyarash/data_0_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_1_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_2_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_3_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_4_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_5_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_6_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_7_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_8_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_9_Keyarash_v7.csv_label_v1.txt', 
'Testing_data/PingweiJiao/data_0_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_1_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_2_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_4_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_5_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_7_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_8_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_9_PingweiJiao_v7.csv_label_v1.txt',  
'Testing_data/Talia/data_1_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_2_Talia_v7.csv_label_v1.txt',
'Testing_data/Talia/data_3_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_4_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_5_Talia_v7.csv_label_v1.txt',
'Testing_data/Talia/data_7_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_9_Talia_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_0_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_1_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_2_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_3_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_4_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_5_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_6_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_7_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_8_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_9_Vineet_v7.csv_label_v1.txt',      
'Testing_data/Song/data_0_Song_v7.csv_label_v1.txt','Testing_data/Song/data_1_Song_v7.csv_label_v1.txt','Testing_data/Song/data_2_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_3_Song_v7.csv_label_v1.txt','Testing_data/Song/data_4_Song_v7.csv_label_v1.txt','Testing_data/Song/data_5_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_6_Song_v7.csv_label_v1.txt','Testing_data/Song/data_7_Song_v7.csv_label_v1.txt','Testing_data/Song/data_8_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_9_Song_v7.csv_label_v1.txt',   
'Testing_data/Vito/data_0_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_1_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_2_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_3_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_4_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_5_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_6_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_7_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_8_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_9_Vito_v7.csv_label_v1.txt',  
'Testing_data/YangLi/data_0_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_1_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_2_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_3_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_4_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_5_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_6_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_7_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_9_YangLi_v7.csv_label_v1.txt',  
'Testing_data/HanTing/data_0_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_1_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_2_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_3_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_4_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_5_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_6_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_7_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_8_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_9_HanTing_v7.csv_label_v1.txt',  
'Testing_data/Naram/data_1_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_2_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_3_Naram_v7.csv_label_v1.txt',
'Testing_data/Naram/data_4_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_5_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_6_Naram_v7.csv_label_v1.txt',
'Testing_data/Naram/data_7_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_8_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_9_Naram_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_0_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_1_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_2_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_3_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_4_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_5_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_6_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_7_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_8_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_9_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_0_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_1_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_2_Weichuan_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_3_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_4_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_5_Weichuan_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_6_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_7_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_8_Weichuan_v7.csv_label_v1.txt',               
'Testing_data/jundaWu/data_0_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_1_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_2_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_3_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_4_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_5_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_6_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_7_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_8_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_9_jundaWu_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_0_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_1_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_2_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_3_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_4_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_5_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_6_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_7_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_8_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_9_shiXun_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_0_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_1_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_2_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_3_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_4_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_5_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_6_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_7_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_8_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_9_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_0_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_3_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_6_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_9_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/liang/data_0_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_1_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_2_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_3_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_4_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_5_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_6_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_7_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_8_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_9_Liang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_0_TingLiang_v7.csv_label_v1.txt', 'Testing_data/TingLiang/data_1_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_2_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_3_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_4_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_5_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_6_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_7_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_8_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_9_TingLiang_v7.csv_label_v1.txt',                                                                                                                                                                                                              
                        ]

### 3.33 load data

In [ ]:
def create_label_frame(file_name):
  label_frame = pd.DataFrame(columns = ['label'])
  for i, file in enumerate(file_name):
    data = pd.read_csv(file, names = ['label'])
    label_frame = pd.concat([label_frame, data], axis = 0, ignore_index = True)
  return label_frame

def create_data_frame(file_name):
  data_frame = []
  for i, file in enumerate(file_name):
    data = pd.read_csv(file)
    data_frame.append (data)
  data_frame = pd.concat(data_frame, join = 'outer')
  return data_frame

def separate_tnt_tst_from_realdata(data_filename, label_filename):
  data_train_frame = []
  data_test_frame = []
  label_train_frame = pd.DataFrame(columns = ['label'])
  label_test_frame = pd.DataFrame(columns = ['label'])
  for i, file in enumerate(data_filename):
    data = pd.read_csv(file)
    label = pd.read_csv(file+'_label_v1.txt', names = ['label'])
    print('data : ', len(data))
    print('label: ', len(label))
    Xtra,Xtst,Ytra,Ytst = train_test_split(data,label,test_size=0.2,shuffle=True)
    data_train_frame.append(Xtra)
    data_test_frame.append(Xtst)
    label_train_frame = pd.concat([label_train_frame, Ytra], axis = 0, ignore_index = True)
    label_test_frame = pd.concat([label_test_frame, Ytst], axis = 0, ignore_index = True)
  data_train_frame = pd.concat(data_train_frame, join = 'outer')
  data_test_frame = pd.concat(data_test_frame, join = 'outer')
  return data_train_frame, data_test_frame, label_train_frame, label_test_frame

def data_3D_transfer(photodiode_arr, data_frame):
  data_3D = list()
  for i, photodiode_name in enumerate(photodiode_arr):
    data_3D.append(data_frame[photodiode_name].values.reshape(-1, ))  
  data_3D = dstack(data_3D)
  data_3D = data_3D.reshape((-1, 300, len(photodiode_arr)))
  return data_3D

def load_dataset(photodiode_arr):
                                         
  label_frame = create_label_frame(label_filenames)
  label_test_frame_new = create_label_frame(label_test_filenames_new)
  
  data_frame = create_data_frame(data_filenames)
  data_test_frame_new = create_data_frame(data_test_filenames_new)
 
  data_3D = data_3D_transfer(photodiode_arr, data_frame)
  data_test_3D_new = data_3D_transfer(photodiode_arr, data_test_frame_new)

  data_train_frame_new, data_test_frame_new2, label_train_frame_new, label_test_frame_new2 = train_test_split(data_test_3D_new, label_test_frame_new)
  label_frame = pd.concat([label_frame, label_train_frame_new], axis = 0, ignore_index = True)
  data_3D = np.concatenate((data_3D, data_train_frame_new), axis = 0)
  data_y = label_frame
  data_test_y_new = label_test_frame_new2  
  print('data_y : ', len(data_y))
  print('data_test_y_new : ', len(data_test_y_new))  
  data_y = to_categorical(data_y,num_classes=10)  
  data_test_y_new = to_categorical(data_test_y_new,num_classes=10)
  Xtra,Xval,Ytra,Yval = train_test_split(data_3D,data_y,test_size=0.2,shuffle=True)
  return  Xtra,Ytra, Xval,Yval,  data_test_frame_new2, data_test_y_new 

## 3.3 Between Subjects

###3.31 trainging_data

In [ ]:

data_filenames = [
          'Training_data/0/data_0_22_v7.csv', 'Training_data/0/data_0_22_2_v7.csv', 'Training_data/0/data_0_121_s1_t1_1_v7.csv', 'Training_data/0/data_0_211_s1_t1_1_v7.csv', 'Training_data/0/data_0_220_s1_t1_1_v7.csv', 'Training_data/0/data_0_221_s1_t1_1_v7.csv',
          'Training_data/0/data_0_221_s1_t2_1_v7.csv', 'Training_data/0/data_0_221_s1_t3_1_v7.csv', 'Training_data/0/data_0_221_s1_t4_3_v7.csv', 'Training_data/0/data_0_221_s1_t6_3_v7.csv', 'Training_data/0/data_0_221_s2_t1_1_v7.csv', 'Training_data/0/data_0_221_s3_t1_1_v7.csv',
          'Training_data/0/data_0_221_s3_t4_1_v7.csv', 'Training_data/0/data_0_221_s3_t5_1_v7.csv', 'Training_data/0/data_0_221_s4_t1_1_v7.csv','Training_data/0/data_0_221_s5_t1_1_v7.csv', 'Training_data/0/data_0_222_s1_t1_1_v7.csv', 'Training_data/0/data_0_222_s1_t2_1_v7.csv', 
          'Training_data/0/data_0_221_s2_t1_1_v7.csv',  

          'Training_data/1/data_1_011_021_v7.csv', 'Training_data/1/data_1_120_122_v7.csv', 'Training_data/1/data_1_021_s1_t1_1_v7.csv', 'Training_data/1/data_1_120_s1_t1_1_v7.csv','Training_data/1/data_1_122_s1_t1_1_v7.csv', 'Training_data/1/data_1_021_s2_t1_1_v7.csv',
          'Training_data/1/data_1_221_s3_t1_1_v7.csv', 'Training_data/1/data_1_221_s4_t1_1_v7.csv', 'Training_data/1/data_1_221_s5_t1_1_v7.csv', 'Training_data/1/data_1_221_s6_t1_1_v7.csv', 'Training_data/1/data_1_221_s3_t1_2_v7.csv', 'Training_data/1/data_1_221_s4_t1_3_v7.csv',
          'Training_data/1/data_1_222_s1_t1_1_v7.csv', 'Training_data/1/data_1_222_s1_t2_1_v7.csv', 'Training_data/1/data_1_222_s2_t1_1_v7.csv',

          'Training_data/2/data_2_22_v7.csv', 'Training_data/2/data_2_22_2_v7.csv', 'Training_data/2/data_2_121_s1_t1_1_v7.csv', 'Training_data/2/data_2_211_s1_t1_1_v7.csv', 'Training_data/2/data_2_220_s1_t1_1_v7.csv','Training_data/2/data_2_221_s1_t1_1_v7.csv',
          'Training_data/2/data_2_222_s1_t1_1_v7.csv', 'Training_data/2/data_2_221_s2_t1_1_v7.csv', 'Training_data/2/data_2_221_s3_t1_1_v7.csv', 'Training_data/2/data_2_221_s3_t1_3_v7.csv','Training_data/2/data_2_222_s2_t1_1_v7.csv', 'Training_data/2/data_2_222_s2_t2_1_v7.csv',
          'Training_data/2/data_2_222_s3_t1_1_v7.csv',

          'Training_data/3/data_3_22_v7.csv','Training_data/3/data_3_22_2_v7.csv','Training_data/3/data_3_121_s1_t1_1_v7.csv','Training_data/3/data_3_121_s2_t1_1_v7.csv', 'Training_data/3/data_3_220_s1_t1_1_v7.csv','Training_data/3/data_3_220_s2_t1_1_v7.csv',
          'Training_data/3/data_3_221_s1_t1_1_v7.csv','Training_data/3/data_3_221_s2_t1_1_v7.csv', 'Training_data/3/data_3_222_s1_t1_1_v7.csv','Training_data/3/data_3_222_s1_t2_1_v7.csv', 'Training_data/3/data_3_222_s2_t1_1_v7.csv','Training_data/3/data_3_222_s2_t2_1_v7.csv',  
                  
          'Training_data/4/data_4_221_v7.csv', 'Training_data/4/data_4_221_2_v7.csv','Training_data/4/data_4_121_s1_t1_1_v7.csv','Training_data/4/data_4_121_s2_t1_1_v7.csv', 'Training_data/4/data_4_211_s1_t1_1_v7.csv', 'Training_data/4/data_4_211_s2_t1_1_v7.csv', 
          'Training_data/4/data_4_210_s1_t1_1_v7.csv','Training_data/4/data_4_210_s2_t1_1_v7.csv','Training_data/4/data_4_220_s1_t1_1_v7.csv','Training_data/4/data_4_220_s2_t1_1_v7.csv','Training_data/4/data_4_221_s1_t1_1_v7.csv','Training_data/4/data_4_221_s2_t1_1_v7.csv',
          'Training_data/4/data_4_221_s2_t2_1_v7.csv','Training_data/4/data_4_221_s2_t1_3_v7.csv','Training_data/4/data_4_221_s2_t2_2_v7.csv','Training_data/4/data_4_221_s3_t1_1_v7.csv','Training_data/4/data_4_221_s4_t1_1_v7.csv','Training_data/4/data_4_221_s5_t1_1_v7.csv',
          'Training_data/4/data_4_221_s6_t1_1_v7.csv','Training_data/4/data_4_221_s6_t1_2_v7.csv','Training_data/4/data_4_221_s6_t1_3_v7.csv','Training_data/4/data_4_222_s1_t1_1_v7.csv','Training_data/4/data_4_222_s1_t2_1_v7.csv','Training_data/4/data_4_222_s2_t1_1_v7.csv',

          'Training_data/5/data_5_221_v7.csv', 'Training_data/5/data_5_221_2_v7.csv', 'Training_data/5/data_5_121_s1_t1_1_v7.csv','Training_data/5/data_5_121_s1_t2_1_v7.csv','Training_data/5/data_5_220_s1_t1_1_v7.csv','Training_data/5/data_5_220_s1_t2_1_v7.csv',
          'Training_data/5/data_5_221_s1_t1_1_v7.csv','Training_data/5/data_5_221_s1_t2_1_v7.csv','Training_data/5/data_5_221_s1_t3_1_v7.csv','Training_data/5/data_5_221_s1_t3_3_v7.csv','Training_data/5/data_5_221_s2_t2_3_v7.csv','Training_data/5/data_5_221_s3_t2_1_v7.csv',
          'Training_data/5/data_5_221_s3_t3_1_v7.csv','Training_data/5/data_5_221_s3_t3_2_v7.csv','Training_data/5/data_5_222_s1_t1_1_v7.csv','Training_data/5/data_5_222_s1_t2_1_v7.csv','Training_data/5/data_5_222_s2_t1_1_v7.csv','Training_data/5/data_5_222_s2_t2_1_v7.csv',

          'Training_data/6/data_6_121_v7.csv', 'Training_data/6/data_6_221_220_v7.csv', 'Training_data/6/data_6_121_s1_t1_1_v7.csv','Training_data/6/data_6_221_s1_t1_1_v7.csv','Training_data/6/data_6_221_s2_t1_1_v7.csv','Training_data/6/data_6_221_s3_t1_1_v7.csv',
          'Training_data/6/data_6_221_s3_t1_3_v7.csv','Training_data/6/data_6_221_s3_t2_1_v7.csv','Training_data/6/data_6_221_s3_t2_2_v7.csv','Training_data/6/data_6_221_s3_t2_3_v7.csv','Training_data/6/data_6_222_s1_t1_1_v7.csv','Training_data/6/data_6_222_s1_t2_1_v7.csv',
          'Training_data/6/data_6_222_s2_t1_1_v7.csv','Training_data/6/data_6_222_s2_t2_1_v7.csv',
                  
          'Training_data/7/data_7_221_v7.csv','Training_data/7/data_7_221_3_v7.csv', 'Training_data/7/data_7_221_2_v7.csv', 'Training_data/7/data_7_121_s1_t1_1_v7.csv','Training_data/7/data_7_121_s2_t1_1_v7.csv','Training_data/7/data_7_121_s3_t1_1_v7.csv', 
          'Training_data/7/data_7_221_s1_t1_1_v7.csv','Training_data/7/data_7_221_s2_t1_1_v7.csv','Training_data/7/data_7_221_s3_t1_1_v7.csv','Training_data/7/data_7_221_s4_t1_1_v7.csv','Training_data/7/data_7_221_s5_t1_1_v7.csv','Training_data/7/data_7_221_s5_t1_3_v7.csv',
          'Training_data/7/data_7_221_s6_t1_1_v7.csv','Training_data/7/data_7_221_s7_t1_1_v7.csv','Training_data/7/data_7_221_s7_t1_3_v7.csv','Training_data/7/data_7_222_s1_t1_1_v7.csv','Training_data/7/data_7_222_s1_t2_1_v7.csv','Training_data/7/data_7_222_s2_t1_1_v7.csv',
          'Training_data/7/data_7_222_s2_t2_1_v7.csv',   
                  
          'Training_data/8/data_8_221_v7.csv', 'Training_data/8/data_8_221_3_v7.csv', 'Training_data/8/data_8_121_s1_t1_1_v7.csv','Training_data/8/data_8_121_s1_t2_1_v7.csv','Training_data/8/data_8_121_s1_t3_1_v7.csv', 'Training_data/8/data_8_220_s1_t1_1_v7.csv',
          'Training_data/8/data_8_220_s1_t2_1_v7.csv','Training_data/8/data_8_220_s1_t3_1_v7.csv','Training_data/8/data_8_221_s1_t1_1_v7.csv','Training_data/8/data_8_221_s1_t2_1_v7.csv','Training_data/8/data_8_221_s1_t3_1_v7.csv','Training_data/8/data_8_221_s1_t4_1_v7.csv',
          'Training_data/8/data_8_221_s1_t5_1_v7.csv','Training_data/8/data_8_121_s1_t4_1_v7.csv','Training_data/8/data_8_221_s1_t6_1_v7.csv','Training_data/8/data_8_221_s1_t7_1_v7.csv','Training_data/8/data_8_221_s1_t7_2_v7.csv','Training_data/8/data_8_221_s1_t7_3_v7.csv',
          'Training_data/8/data_8_221_s1_t8_1_v7.csv','Training_data/8/data_8_221_s1_t8_3_v7.csv','Training_data/8/data_8_221_s2_t1_1_v7.csv','Training_data/8/data_8_222_s1_t1_1_v7.csv','Training_data/8/data_8_222_s1_t2_1_v7.csv','Training_data/8/data_8_222_s2_t1_1_v7.csv',
                  
          'Training_data/9/data_9_120_v7.csv', 'Training_data/9/data_9_221_220_v7.csv','Training_data/9/data_9_221_220_3_v7.csv','Training_data/9/data_9_120_s1_t1_1_v7.csv','Training_data/9/data_9_121_s1_t1_1_v7.csv','Training_data/9/data_9_220_s1_t1_1_v7.csv',
          'Training_data/9/data_9_221_s1_t1_1_v7.csv','Training_data/9/data_9_220_s1_t2_1_v7.csv','Training_data/9/data_9_221_s1_t2_2_v7.csv','Training_data/9/data_9_221_s1_t2_3_v7.csv','Training_data/9/data_9_221_s2_t1_1_v7.csv','Training_data/9/data_9_221_s3_t1_1_v7.csv',
          'Training_data/9/data_9_221_s3_t1_2_v7.csv' ,'Training_data/9/data_9_221_s3_t1_3_v7.csv' ,'Training_data/9/data_9_221_s4_t1_3_v7.csv','Training_data/9/data_9_222_s1_t1_1_v7.csv','Training_data/9/data_9_222_s1_t2_1_v7.csv','Training_data/9/data_9_222_s2_t1_1_v7.csv',
                   
          ]


label_filenames = [
'Training_data/0/data_0_22_v7.csv_label_v1.txt','Training_data/0/data_0_22_2_v7.csv_label_v1.txt','Training_data/0/data_0_121_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_211_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_220_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t1_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/0/data_0_221_s1_t3_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t4_3_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s1_t6_3_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s2_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s3_t1_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s3_t4_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s3_t5_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/0/data_0_221_s5_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_222_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/0/data_0_222_s1_t2_1_v7.csv_label_v1.txt', 
'Training_data/0/data_0_221_s2_t1_1_v7.csv_label_v1.txt', 

'Training_data/1/data_1_011_021_v7.csv_label_v1.txt','Training_data/1/data_1_120_122_v7.csv_label_v1.txt', 'Training_data/1/data_1_021_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_120_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_122_s1_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_021_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/1/data_1_221_s3_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s5_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s6_t1_1_v7.csv_label_v1.txt','Training_data/1/data_1_221_s3_t1_2_v7.csv_label_v1.txt','Training_data/1/data_1_221_s4_t1_3_v7.csv_label_v1.txt',
'Training_data/1/data_1_222_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/1/data_1_222_s1_t2_1_v7.csv_label_v1.txt', 'Training_data/1/data_1_222_s2_t1_1_v7.csv_label_v1.txt',

'Training_data/2/data_2_22_v7.csv_label_v1.txt','Training_data/2/data_2_22_2_v7.csv_label_v1.txt','Training_data/2/data_2_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_211_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/2/data_2_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/2/data_2_221_s3_t1_1_v7.csv_label_v1.txt', 'Training_data/2/data_2_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/2/data_2_222_s2_t1_1_v7.csv_label_v1.txt', 'Training_data/2/data_2_222_s2_t2_1_v7.csv_label_v1.txt',
'Training_data/2/data_2_222_s3_t1_1_v7.csv_label_v1.txt', 
                  
'Training_data/3/data_3_22_v7.csv_label_v1.txt','Training_data/3/data_3_22_2_v7.csv_label_v1.txt','Training_data/3/data_3_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_220_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/3/data_3_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s1_t2_1_v7.csv_label_v1.txt', 'Training_data/3/data_3_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/3/data_3_222_s2_t2_1_v7.csv_label_v1.txt',  
                  
'Training_data/4/data_4_221_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_2_v7.csv_label_v1.txt','Training_data/4/data_4_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_211_s1_t1_1_v7.csv_label_v1.txt', 'Training_data/4/data_4_211_s2_t1_1_v7.csv_label_v1.txt', 
'Training_data/4/data_4_210_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_210_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_220_s2_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/4/data_4_221_s2_t2_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t1_3_v7.csv_label_v1.txt','Training_data/4/data_4_221_s2_t2_2_v7.csv_label_v1.txt','Training_data/4/data_4_221_s3_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s4_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_221_s5_t1_1_v7.csv_label_v1.txt',
'Training_data/4/data_4_221_s6_t1_1_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_s6_t1_2_v7.csv_label_v1.txt', 'Training_data/4/data_4_221_s6_t1_3_v7.csv_label_v1.txt','Training_data/4/data_4_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/4/data_4_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/4/data_4_222_s2_t1_1_v7.csv_label_v1.txt', 
                  
'Training_data/5/data_5_221_v7.csv_label_v1.txt', 'Training_data/5/data_5_221_2_v7.csv_label_v1.txt', 'Training_data/5/data_5_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_121_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_220_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_220_s1_t2_1_v7.csv_label_v1.txt',
'Training_data/5/data_5_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t3_1_v7.csv_label_v1.txt','Training_data/5/data_5_221_s1_t3_3_v7.csv_label_v1.txt','Training_data/5/data_5_221_s2_t2_3_v7.csv_label_v1.txt','Training_data/5/data_5_221_s3_t2_1_v7.csv_label_v1.txt',
'Training_data/5/data_5_221_s3_t3_1_v7.csv_label_v1.txt', 'Training_data/5/data_5_221_s3_t3_2_v7.csv_label_v1.txt','Training_data/5/data_5_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/5/data_5_222_s2_t2_1_v7.csv_label_v1.txt', 
                  
'Training_data/6/data_6_121_v7.csv_label_v1.txt', 'Training_data/6/data_6_221_220_v7.csv_label_v1.txt', 'Training_data/6/data_6_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t1_1_v7.csv_label_v1.txt',
'Training_data/6/data_6_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_1_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_2_v7.csv_label_v1.txt','Training_data/6/data_6_221_s3_t2_3_v7.csv_label_v1.txt','Training_data/6/data_6_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_222_s1_t2_1_v7.csv_label_v1.txt',
'Training_data/6/data_6_222_s2_t1_1_v7.csv_label_v1.txt','Training_data/6/data_6_222_s2_t2_1_v7.csv_label_v1.txt',
                  
'Training_data/7/data_7_221_v7.csv_label_v1.txt','Training_data/7/data_7_221_3_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_2_v7.csv_label_v1.txt', 'Training_data/7/data_7_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_121_s2_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_121_s3_t1_1_v7.csv_label_v1.txt', 
'Training_data/7/data_7_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_221_s3_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s4_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s5_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s5_t1_3_v7.csv_label_v1.txt', 
'Training_data/7/data_7_221_s6_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s7_t1_1_v7.csv_label_v1.txt', 'Training_data/7/data_7_221_s7_t1_3_v7.csv_label_v1.txt','Training_data/7/data_7_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/7/data_7_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/7/data_7_222_s2_t1_1_v7.csv_label_v1.txt',
'Training_data/7/data_7_222_s2_t2_1_v7.csv_label_v1.txt',   
                  
'Training_data/8/data_8_221_v7.csv_label_v1.txt', 'Training_data/8/data_8_221_3_v7.csv_label_v1.txt', 'Training_data/8/data_8_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t3_1_v7.csv_label_v1.txt', 'Training_data/8/data_8_220_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/8/data_8_220_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_220_s1_t3_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t3_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t4_1_v7.csv_label_v1.txt',
'Training_data/8/data_8_221_s1_t5_1_v7.csv_label_v1.txt','Training_data/8/data_8_121_s1_t4_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t6_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_2_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t7_3_v7.csv_label_v1.txt',
'Training_data/8/data_8_221_s1_t8_1_v7.csv_label_v1.txt','Training_data/8/data_8_221_s1_t8_3_v7.csv_label_v1.txt','Training_data/8/data_8_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/8/data_8_222_s2_t1_1_v7.csv_label_v1.txt',
                  
'Training_data/9/data_9_120_v7.csv_label_v1.txt', 'Training_data/9/data_9_221_220_v7.csv_label_v1.txt','Training_data/9/data_9_221_220_3_v7.csv_label_v1.txt','Training_data/9/data_9_120_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_121_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_220_s1_t1_1_v7.csv_label_v1.txt',
'Training_data/9/data_9_221_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_220_s1_t2_1_v7.csv_label_v1.txt','Training_data/9/data_9_221_s1_t2_2_v7.csv_label_v1.txt','Training_data/9/data_9_221_s1_t2_3_v7.csv_label_v1.txt','Training_data/9/data_9_221_s2_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_221_s3_t1_1_v7.csv_label_v1.txt',
'Training_data/9/data_9_221_s3_t1_2_v7.csv_label_v1.txt','Training_data/9/data_9_221_s3_t1_3_v7.csv_label_v1.txt','Training_data/9/data_9_221_s4_t1_3_v7.csv_label_v1.txt','Training_data/9/data_9_222_s1_t1_1_v7.csv_label_v1.txt','Training_data/9/data_9_222_s1_t2_1_v7.csv_label_v1.txt','Training_data/9/data_9_222_s2_t1_1_v7.csv_label_v1.txt',
                
                              
          ]
   

### 3.32 testing_data

In [ ]:
data_test_filenames_new = [
'Testing_data/HaozheTang/data_0_HaozheTang_v7.csv','Testing_data/HaozheTang/data_1_HaozheTang_v7.csv','Testing_data/HaozheTang/data_2_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_3_HaozheTang_v7.csv','Testing_data/HaozheTang/data_4_HaozheTang_v7.csv','Testing_data/HaozheTang/data_5_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_6_HaozheTang_v7.csv','Testing_data/HaozheTang/data_7_HaozheTang_v7.csv','Testing_data/HaozheTang/data_8_HaozheTang_v7.csv',
'Testing_data/HaozheTang/data_9_HaozheTang_v7.csv',
'Testing_data/Qing/data_0_Qing_v7.csv', 'Testing_data/Qing/data_1_Qing_v7.csv', 'Testing_data/Qing/data_2_Qing_v7.csv',
'Testing_data/Qing/data_3_Qing_v7.csv', 'Testing_data/Qing/data_4_Qing_v7.csv', 'Testing_data/Qing/data_5_Qing_v7.csv',
'Testing_data/Qing/data_6_Qing_v7.csv', 'Testing_data/Qing/data_7_Qing_v7.csv', 'Testing_data/Qing/data_8_Qing_v7.csv',
'Testing_data/Qing/data_9_Qing_v7.csv',
'Testing_data/Serge/data_0_Serge_v7.csv','Testing_data/Serge/data_1_Serge_v7.csv','Testing_data/Serge/data_2_Serge_v7.csv',
'Testing_data/Serge/data_3_Serge_v7.csv','Testing_data/Serge/data_4_Serge_v7.csv','Testing_data/Serge/data_5_Serge_v7.csv',
'Testing_data/Serge/data_6_Serge_v7.csv','Testing_data/Serge/data_7_Serge_v7.csv','Testing_data/Serge/data_8_Serge_v7.csv',
'Testing_data/Serge/data_9_Serge_v7.csv',
'Testing_data/AlexCui/data_0_AlexCui_v7.csv','Testing_data/AlexCui/data_1_AlexCui_v7.csv','Testing_data/AlexCui/data_2_AlexCui_v7.csv',
'Testing_data/AlexCui/data_3_AlexCui_v7.csv','Testing_data/AlexCui/data_4_AlexCui_v7.csv','Testing_data/AlexCui/data_5_AlexCui_v7.csv',
'Testing_data/AlexCui/data_6_AlexCui_v7.csv','Testing_data/AlexCui/data_7_AlexCui_v7.csv','Testing_data/AlexCui/data_8_AlexCui_v7.csv',
'Testing_data/AlexCui/data_9_AlexCui_v7.csv', 
'Testing_data/Keyarash/data_0_Keyarash_v7.csv','Testing_data/Keyarash/data_1_Keyarash_v7.csv','Testing_data/Keyarash/data_2_Keyarash_v7.csv',
'Testing_data/Keyarash/data_3_Keyarash_v7.csv','Testing_data/Keyarash/data_4_Keyarash_v7.csv','Testing_data/Keyarash/data_5_Keyarash_v7.csv',
'Testing_data/Keyarash/data_6_Keyarash_v7.csv','Testing_data/Keyarash/data_7_Keyarash_v7.csv','Testing_data/Keyarash/data_8_Keyarash_v7.csv',
'Testing_data/Keyarash/data_9_Keyarash_v7.csv',
'Testing_data/PingweiJiao/data_0_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_1_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_2_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_4_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_5_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_7_PingweiJiao_v7.csv','Testing_data/PingweiJiao/data_8_PingweiJiao_v7.csv',
'Testing_data/PingweiJiao/data_9_PingweiJiao_v7.csv',                        
'Testing_data/Talia/data_1_Talia_v7.csv','Testing_data/Talia/data_2_Talia_v7.csv',
'Testing_data/Talia/data_3_Talia_v7.csv','Testing_data/Talia/data_4_Talia_v7.csv','Testing_data/Talia/data_5_Talia_v7.csv',
'Testing_data/Talia/data_7_Talia_v7.csv',
'Testing_data/Talia/data_9_Talia_v7.csv',
'Testing_data/Vineet/data_0_Vineet_v7.csv','Testing_data/Vineet/data_1_Vineet_v7.csv','Testing_data/Vineet/data_2_Vineet_v7.csv',
'Testing_data/Vineet/data_3_Vineet_v7.csv','Testing_data/Vineet/data_4_Vineet_v7.csv','Testing_data/Vineet/data_5_Vineet_v7.csv',
'Testing_data/Vineet/data_6_Vineet_v7.csv','Testing_data/Vineet/data_7_Vineet_v7.csv','Testing_data/Vineet/data_8_Vineet_v7.csv',
'Testing_data/Vineet/data_9_Vineet_v7.csv',   
'Testing_data/Song/data_0_Song_v7.csv','Testing_data/Song/data_1_Song_v7.csv','Testing_data/Song/data_2_Song_v7.csv',
'Testing_data/Song/data_3_Song_v7.csv','Testing_data/Song/data_4_Song_v7.csv','Testing_data/Song/data_5_Song_v7.csv',
'Testing_data/Song/data_6_Song_v7.csv','Testing_data/Song/data_7_Song_v7.csv','Testing_data/Song/data_8_Song_v7.csv',
'Testing_data/Song/data_9_Song_v7.csv',   
'Testing_data/Vito/data_0_Vito_v7.csv','Testing_data/Vito/data_1_Vito_v7.csv','Testing_data/Vito/data_2_Vito_v7.csv',
'Testing_data/Vito/data_3_Vito_v7.csv','Testing_data/Vito/data_4_Vito_v7.csv','Testing_data/Vito/data_5_Vito_v7.csv',
'Testing_data/Vito/data_6_Vito_v7.csv','Testing_data/Vito/data_7_Vito_v7.csv','Testing_data/Vito/data_8_Vito_v7.csv',
'Testing_data/Vito/data_9_Vito_v7.csv', 
'Testing_data/YangLi/data_0_YangLi_v7.csv','Testing_data/YangLi/data_1_YangLi_v7.csv','Testing_data/YangLi/data_2_YangLi_v7.csv',
'Testing_data/YangLi/data_3_YangLi_v7.csv','Testing_data/YangLi/data_4_YangLi_v7.csv','Testing_data/YangLi/data_5_YangLi_v7.csv',
'Testing_data/YangLi/data_6_YangLi_v7.csv','Testing_data/YangLi/data_7_YangLi_v7.csv',
'Testing_data/YangLi/data_9_YangLi_v7.csv',
'Testing_data/HanTing/data_0_HanTing_v7.csv','Testing_data/HanTing/data_1_HanTing_v7.csv','Testing_data/HanTing/data_2_HanTing_v7.csv',
'Testing_data/HanTing/data_3_HanTing_v7.csv','Testing_data/HanTing/data_4_HanTing_v7.csv','Testing_data/HanTing/data_5_HanTing_v7.csv',
'Testing_data/HanTing/data_6_HanTing_v7.csv','Testing_data/HanTing/data_7_HanTing_v7.csv','Testing_data/HanTing/data_8_HanTing_v7.csv',
'Testing_data/HanTing/data_9_HanTing_v7.csv',
'Testing_data/Naram/data_1_Naram_v7.csv','Testing_data/Naram/data_2_Naram_v7.csv','Testing_data/Naram/data_3_Naram_v7.csv',
'Testing_data/Naram/data_4_Naram_v7.csv','Testing_data/Naram/data_5_Naram_v7.csv','Testing_data/Naram/data_6_Naram_v7.csv',
'Testing_data/Naram/data_7_Naram_v7.csv','Testing_data/Naram/data_8_Naram_v7.csv','Testing_data/Naram/data_9_Naram_v7.csv',   
'Testing_data/Ricardo/data_0_Ricardo_v7.csv','Testing_data/Ricardo/data_1_Ricardo_v7.csv','Testing_data/Ricardo/data_2_Ricardo_v7.csv',
'Testing_data/Ricardo/data_3_Ricardo_v7.csv','Testing_data/Ricardo/data_4_Ricardo_v7.csv','Testing_data/Ricardo/data_5_Ricardo_v7.csv',
'Testing_data/Ricardo/data_6_Ricardo_v7.csv','Testing_data/Ricardo/data_7_Ricardo_v7.csv','Testing_data/Ricardo/data_8_Ricardo_v7.csv',
'Testing_data/Ricardo/data_9_Ricardo_v7.csv',
'Testing_data/Weichuan/data_0_Weichuan_v7.csv','Testing_data/Weichuan/data_1_Weichuan_v7.csv','Testing_data/Weichuan/data_2_Weichuan_v7.csv',
'Testing_data/Weichuan/data_3_Weichuan_v7.csv','Testing_data/Weichuan/data_4_Weichuan_v7.csv','Testing_data/Weichuan/data_5_Weichuan_v7.csv',
'Testing_data/Weichuan/data_6_Weichuan_v7.csv','Testing_data/Weichuan/data_7_Weichuan_v7.csv','Testing_data/Weichuan/data_8_Weichuan_v7.csv',       
'Testing_data/jundaWu/data_0_jundaWu_v7.csv','Testing_data/jundaWu/data_1_jundaWu_v7.csv','Testing_data/jundaWu/data_2_jundaWu_v7.csv',
'Testing_data/jundaWu/data_3_jundaWu_v7.csv','Testing_data/jundaWu/data_4_jundaWu_v7.csv','Testing_data/jundaWu/data_5_jundaWu_v7.csv',
'Testing_data/jundaWu/data_6_jundaWu_v7.csv','Testing_data/jundaWu/data_7_jundaWu_v7.csv','Testing_data/jundaWu/data_8_jundaWu_v7.csv',
'Testing_data/jundaWu/data_9_jundaWu_v7.csv',
'Testing_data/shixunWu/data_0_shiXun_v7.csv','Testing_data/shixunWu/data_1_shiXun_v7.csv','Testing_data/shixunWu/data_2_shiXun_v7.csv',
'Testing_data/shixunWu/data_3_shiXun_v7.csv','Testing_data/shixunWu/data_4_shiXun_v7.csv','Testing_data/shixunWu/data_5_shiXun_v7.csv',
'Testing_data/shixunWu/data_6_shiXun_v7.csv','Testing_data/shixunWu/data_7_shiXun_v7.csv','Testing_data/shixunWu/data_8_shiXun_v7.csv',
'Testing_data/shixunWu/data_9_shiXun_v7.csv',
'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv',
'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv',
'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv',
'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv',
'Testing_data/shuxinChen/data_0_shuxinChen_v7.csv','Testing_data/shuxinChen/data_1_shuxinChen_v7.csv','Testing_data/shuxinChen/data_2_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_3_shuxinChen_v7.csv','Testing_data/shuxinChen/data_4_shuxinChen_v7.csv','Testing_data/shuxinChen/data_5_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_6_shuxinChen_v7.csv','Testing_data/shuxinChen/data_7_shuxinChen_v7.csv','Testing_data/shuxinChen/data_8_shuxinChen_v7.csv',
'Testing_data/shuxinChen/data_9_shuxinChen_v7.csv',
'Testing_data/zehangWu/data_0_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_3_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_6_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv',
'Testing_data/zehangWu/data_9_ZehangWu_v7.csv',
'Testing_data/liang/data_0_Liang_v7.csv','Testing_data/liang/data_1_Liang_v7.csv','Testing_data/liang/data_2_Liang_v7.csv',
'Testing_data/liang/data_3_Liang_v7.csv','Testing_data/liang/data_4_Liang_v7.csv','Testing_data/liang/data_5_Liang_v7.csv',
'Testing_data/liang/data_6_Liang_v7.csv','Testing_data/liang/data_7_Liang_v7.csv','Testing_data/liang/data_8_Liang_v7.csv',
'Testing_data/liang/data_9_Liang_v7.csv',
'Testing_data/TingLiang/data_0_TingLiang_v7.csv', 'Testing_data/TingLiang/data_1_TingLiang_v7.csv','Testing_data/TingLiang/data_2_TingLiang_v7.csv',
'Testing_data/TingLiang/data_3_TingLiang_v7.csv','Testing_data/TingLiang/data_4_TingLiang_v7.csv','Testing_data/TingLiang/data_5_TingLiang_v7.csv',
'Testing_data/TingLiang/data_6_TingLiang_v7.csv','Testing_data/TingLiang/data_7_TingLiang_v7.csv','Testing_data/TingLiang/data_8_TingLiang_v7.csv',
'Testing_data/TingLiang/data_9_TingLiang_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_0_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_1_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_2_ZehangWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_4_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_5_ZehangWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_7_ZehangWu_v7.csv', 'Testing_data3/Mid/Hao/data_8_ZehangWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_ZehangWu_v7.csv', 
                        # 'Testing_data3/Mid/Hao/data_0_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_1_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_2_Hao6_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_4_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_5_Hao6_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_7_Hao6_v7.csv', 'Testing_data3/Mid/Hao/data_8_Hao6_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_Hao6_v7.csv', 
                        # 'Testing_data3/Mid/Hao/data_0_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_1_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_2_Hao7_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_4_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_5_Hao7_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_7_Hao7_v7.csv', 'Testing_data3/Mid/Hao/data_8_Hao7_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_Hao7_v7.csv',                         
                        # 'Testing_data3/Mid/Hao/data_0_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_1_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_2_Liangting_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_4_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_5_Liangting_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_7_Liangting_v7.csv', 'Testing_data3/Mid/Hao/data_8_Liangting_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_Liangting_v7.csv', #26%
                        # 'Testing_data3/Mid/Hao/data_0_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_1_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_2_ShuxinChen_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_4_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_5_ShuxinChen_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_7_ShuxinChen_v7.csv', 'Testing_data3/Mid/Hao/data_8_ShuxinChen_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_ShuxinChen_v7.csv', #17.5
                        # 'Testing_data3/Mid/Hao/data_0_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_1_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_2_TeacherWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_4_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_5_TeacherWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_7_TeacherWu_v7.csv', 'Testing_data3/Mid/Hao/data_8_TeacherWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_TeacherWu_v7.csv',  #13.1
                        # 'Testing_data3/Mid/Hao/data_0_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_1_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_2_Yay_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_4_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_5_Yay_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_7_Yay_v7.csv', 'Testing_data3/Mid/Hao/data_8_Yay_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_Yay_v7.csv', #13.1
                        # 'Testing_data3/Mid/Hao/data_0_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_1_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_2_ShixunWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_3_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_4_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_5_ShixunWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_6_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_7_ShixunWu_v7.csv', 'Testing_data3/Mid/Hao/data_8_ShixunWu_v7.csv',
                        # 'Testing_data3/Mid/Hao/data_9_ShixunWu_v7.csv', #34                        

                        ]
label_test_filenames_new = [
'Testing_data/HaozheTang/data_0_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_1_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_2_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_3_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_4_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_5_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_6_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_7_HaozheTang_v7.csv_label_v1.txt','Testing_data/HaozheTang/data_8_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/HaozheTang/data_9_HaozheTang_v7.csv_label_v1.txt',
'Testing_data/Qing/data_0_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_1_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_2_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_3_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_4_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_5_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_6_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_7_Qing_v7.csv_label_v1.txt', 'Testing_data/Qing/data_8_Qing_v7.csv_label_v1.txt',
'Testing_data/Qing/data_9_Qing_v7.csv_label_v1.txt',
'Testing_data/Serge/data_0_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_1_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_2_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_3_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_4_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_5_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_6_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_7_Serge_v7.csv_label_v1.txt','Testing_data/Serge/data_8_Serge_v7.csv_label_v1.txt',
'Testing_data/Serge/data_9_Serge_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_0_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_1_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_2_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_3_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_4_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_5_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_6_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_7_AlexCui_v7.csv_label_v1.txt','Testing_data/AlexCui/data_8_AlexCui_v7.csv_label_v1.txt',
'Testing_data/AlexCui/data_9_AlexCui_v7.csv_label_v1.txt',  
'Testing_data/Keyarash/data_0_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_1_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_2_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_3_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_4_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_5_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_6_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_7_Keyarash_v7.csv_label_v1.txt','Testing_data/Keyarash/data_8_Keyarash_v7.csv_label_v1.txt',
'Testing_data/Keyarash/data_9_Keyarash_v7.csv_label_v1.txt', 
'Testing_data/PingweiJiao/data_0_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_1_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_2_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_4_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_5_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_7_PingweiJiao_v7.csv_label_v1.txt','Testing_data/PingweiJiao/data_8_PingweiJiao_v7.csv_label_v1.txt',
'Testing_data/PingweiJiao/data_9_PingweiJiao_v7.csv_label_v1.txt',  
'Testing_data/Talia/data_1_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_2_Talia_v7.csv_label_v1.txt',
'Testing_data/Talia/data_3_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_4_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_5_Talia_v7.csv_label_v1.txt',
'Testing_data/Talia/data_7_Talia_v7.csv_label_v1.txt','Testing_data/Talia/data_9_Talia_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_0_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_1_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_2_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_3_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_4_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_5_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_6_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_7_Vineet_v7.csv_label_v1.txt','Testing_data/Vineet/data_8_Vineet_v7.csv_label_v1.txt',
'Testing_data/Vineet/data_9_Vineet_v7.csv_label_v1.txt',      
'Testing_data/Song/data_0_Song_v7.csv_label_v1.txt','Testing_data/Song/data_1_Song_v7.csv_label_v1.txt','Testing_data/Song/data_2_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_3_Song_v7.csv_label_v1.txt','Testing_data/Song/data_4_Song_v7.csv_label_v1.txt','Testing_data/Song/data_5_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_6_Song_v7.csv_label_v1.txt','Testing_data/Song/data_7_Song_v7.csv_label_v1.txt','Testing_data/Song/data_8_Song_v7.csv_label_v1.txt',
'Testing_data/Song/data_9_Song_v7.csv_label_v1.txt',   
'Testing_data/Vito/data_0_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_1_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_2_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_3_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_4_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_5_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_6_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_7_Vito_v7.csv_label_v1.txt','Testing_data/Vito/data_8_Vito_v7.csv_label_v1.txt',
'Testing_data/Vito/data_9_Vito_v7.csv_label_v1.txt',  
'Testing_data/YangLi/data_0_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_1_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_2_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_3_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_4_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_5_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_6_YangLi_v7.csv_label_v1.txt','Testing_data/YangLi/data_7_YangLi_v7.csv_label_v1.txt',
'Testing_data/YangLi/data_9_YangLi_v7.csv_label_v1.txt',  
'Testing_data/HanTing/data_0_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_1_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_2_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_3_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_4_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_5_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_6_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_7_HanTing_v7.csv_label_v1.txt','Testing_data/HanTing/data_8_HanTing_v7.csv_label_v1.txt',
'Testing_data/HanTing/data_9_HanTing_v7.csv_label_v1.txt',  
'Testing_data/Naram/data_1_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_2_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_3_Naram_v7.csv_label_v1.txt',
'Testing_data/Naram/data_4_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_5_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_6_Naram_v7.csv_label_v1.txt',
'Testing_data/Naram/data_7_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_8_Naram_v7.csv_label_v1.txt','Testing_data/Naram/data_9_Naram_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_0_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_1_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_2_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_3_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_4_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_5_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_6_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_7_Ricardo_v7.csv_label_v1.txt','Testing_data/Ricardo/data_8_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Ricardo/data_9_Ricardo_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_0_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_1_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_2_Weichuan_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_3_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_4_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_5_Weichuan_v7.csv_label_v1.txt',
'Testing_data/Weichuan/data_6_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_7_Weichuan_v7.csv_label_v1.txt','Testing_data/Weichuan/data_8_Weichuan_v7.csv_label_v1.txt',               
'Testing_data/jundaWu/data_0_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_1_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_2_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_3_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_4_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_5_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_6_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_7_jundaWu_v7.csv_label_v1.txt','Testing_data/jundaWu/data_8_jundaWu_v7.csv_label_v1.txt',
'Testing_data/jundaWu/data_9_jundaWu_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_0_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_1_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_2_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_3_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_4_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_5_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_6_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_7_shiXun_v7.csv_label_v1.txt','Testing_data/shixunWu/data_8_shiXun_v7.csv_label_v1.txt',
'Testing_data/shixunWu/data_9_shiXun_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_0_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_1_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_2_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_3_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_4_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_5_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_6_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_7_XinyunXu_v7.csv_label_v1.txt','Testing_data/xinyunXu/data_8_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/xinyunXu/data_9_XinyunXu_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_0_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_1_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_2_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_3_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_4_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_5_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_6_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_7_xie_3_v7.csv_label_v1.txt','Testing_data/yuanhaoXie/data_8_xie_3_v7.csv_label_v1.txt',
'Testing_data/yuanhaoXie/data_9_xie_3_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_0_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_1_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_2_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_3_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_4_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_5_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_6_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_7_shuxinChen_v7.csv_label_v1.txt','Testing_data/shuxinChen/data_8_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/shuxinChen/data_9_shuxinChen_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_0_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_1_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_2_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_3_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_4_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_5_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_6_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_7_ZehangWu_v7.csv_label_v1.txt', 'Testing_data/zehangWu/data_8_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/zehangWu/data_9_ZehangWu_v7.csv_label_v1.txt',
'Testing_data/liang/data_0_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_1_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_2_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_3_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_4_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_5_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_6_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_7_Liang_v7.csv_label_v1.txt','Testing_data/liang/data_8_Liang_v7.csv_label_v1.txt',
'Testing_data/liang/data_9_Liang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_0_TingLiang_v7.csv_label_v1.txt', 'Testing_data/TingLiang/data_1_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_2_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_3_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_4_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_5_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_6_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_7_TingLiang_v7.csv_label_v1.txt','Testing_data/TingLiang/data_8_TingLiang_v7.csv_label_v1.txt',
'Testing_data/TingLiang/data_9_TingLiang_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_ZehangWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_ZehangWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_ZehangWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_ZehangWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_ZehangWu_v7.csv_label_v1.txt',  
                        # 'Testing_data3/Mid/Hao/data_0_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_Hao6_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_Hao6_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_Hao6_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_Hao6_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_Hao6_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_Hao7_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_Hao7_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_Hao7_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_Hao7_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_Hao7_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_Liangting_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_Liangting_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_Liangting_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_Liangting_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_Liangting_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_ShuxinChen_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_ShuxinChen_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_ShuxinChen_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_ShuxinChen_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_ShuxinChen_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_TeacherWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_TeacherWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_TeacherWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_TeacherWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_TeacherWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_Yay_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_Yay_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_Yay_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_Yay_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_Yay_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_0_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_1_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_2_ShixunWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_3_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_4_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_5_ShixunWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_6_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_7_ShixunWu_v7.csv_label_v1.txt', 'Testing_data3/Mid/Hao/data_8_ShixunWu_v7.csv_label_v1.txt',
                        # 'Testing_data3/Mid/Hao/data_9_ShixunWu_v7.csv_label_v1.txt',                                                                                                                                                                                                              
                        ]

### 3.33 load data

In [ ]:
def data_3D_transfer(photodiode_arr, data_frame):
    data_3D = list()
    num_fea = len(photodiode_arr)
    for i, photodiode_name in enumerate(photodiode_arr):
      data_3D.append(data_frame[photodiode_name].values.reshape(-1, ))  
    data_3D = dstack(data_3D)
    data_3D = data_3D.reshape((-1, 300, num_fea))
    return data_3D

def cross_validation_create_data_label_frame(data_test_filenames_new, label_test_filenames_new, trainName, testName, data_filenames, label_filenames ):
    data_train_frame = []
    data_test_frame = []
    label_train_frame = pd.DataFrame(columns=['label'])
    label_test_frame = pd.DataFrame(columns=['label'])   
    
    for i, file in enumerate(data_filenames):
        data = pd.read_csv(file)
        label = pd.read_csv(label_filenames[i], names=['label'])
        data_train_frame.append(data)
        label_train_frame = pd.concat([label_train_frame, label], axis= 0, ignore_index=True)
    
    for i, filename in enumerate (data_test_filenames_new):
        split_name = filename.split('_')
        data = pd.read_csv(filename)
        label = pd.read_csv(label_test_filenames_new[i], names=['label'])
        if split_name[-2] in trainName:
            data_train_frame.append(data)
            label_train_frame = pd.concat([label_train_frame, label], axis= 0, ignore_index=True)
        elif split_name[-2] in testName:
            data_test_frame.append(data) 
            label_test_frame = pd.concat([label_test_frame, label], axis= 0, ignore_index=True)
        else:
            data_test_frame.append(data)
            label_test_frame = pd.concat([label_test_frame, label], axis= 0, ignore_index=True)
    print('len data_train_frame : ', len(data_train_frame))
    print('len data_test_frame : ', len(data_test_frame))
    data_train_frame = pd.concat(data_train_frame, join = 'outer')
    data_test_frame = pd.concat(data_test_frame, join = 'outer')
    return data_train_frame, label_train_frame, data_test_frame, label_test_frame


def load_dataset(data_test_filenames_new, label_test_filenames_new, trainName, testName, data_filenames, label_filenames, photodiode_arr):
    data_train_frame, label_train_frame, data_test_frame, label_test_frame = cross_validation_create_data_label_frame(data_test_filenames_new, label_test_filenames_new, trainName, testName, data_filenames, label_filenames)
    
    data_3D = data_3D_transfer(photodiode_arr, data_train_frame)
    data_test_3D_new = data_3D_transfer(photodiode_arr, data_test_frame)

    data_y = label_train_frame
    data_test_y_new = label_test_frame  

    print('data_y : ', len(data_y))
    print('data_test_y_new : ', len(data_test_y_new))  
    
    data_y = to_categorical(data_y,num_classes=10)  
    data_test_y_new = to_categorical(data_test_y_new,num_classes=10)

    Xtra,Xval,Ytra,Yval = train_test_split(data_3D,data_y,test_size=0.2,shuffle=True)
    return  Xtra,Ytra, Xval,Yval,  data_test_3D_new, data_test_y_new 

# 4. Setting hyperparameters

In [ ]:
learning_rate = [ 0.001]
verbose, epochs  = 0, 20
Batch_size = [128]

n_steps, n_length = 60, 5
num_filters = [64] #64
num_kernels = [3]
num_lstm_Neurons = [512]
num_fcn_Neurons = [512]
Attention_Neurons = [1]


# 5. Building models

## 5.1 CNN- BiGRU - selfAtt - Flatten(GAP) - FCN

In [ ]:
def build_model(num_head, att_neuron, num_lstm_Neuron, num_fcn_Neuron, num_filter,num_kernel, n_timesteps, n_steps, n_length, n_features, n_outputs):
  inputs = Input(shape = (n_steps, n_length, n_features))
  TDCnn = TimeDistributed(Conv1D(filters= num_filter, kernel_size=num_kernel,  activation = 'relu'))(inputs)
  Pool = TimeDistributed(MaxPooling1D(pool_size =num_kernel))(TDCnn)  
  dp = TimeDistributed(Dropout(0.5))(Pool)
  flat = TimeDistributed(Flatten())(dp)
  gru = GRU(num_lstm_Neuron, return_sequences = False)(flat)
  dp2 = Dropout(0.5)(gru)
  sAtt = SeqSelfAttention(attention_activation='sigmoid')(dp2)
  fla = GlobalAveragePooling1D()(sAtt)
  den = Dense(num_fcn_Neuron, activation='relu')(fla)
  output = Dense(n_outputs, activation='softmax')(den)
  model = Model(inputs = inputs, outputs = output) 
  return model  


# 6. Evaluating models

In [ ]:
def evaluate_model(num_head, trainX, trainy, testX, testy, valX, valy,learningRate,batch_size,num_filter, num_kernel, att_neurons, num_lstm_Neuron, num_fcn_Neuron, n_timesteps, n_steps, n_length,n_features, n_outputs):
  model = build_model(num_head, att_neurons, num_lstm_Neuron, num_fcn_Neuron,num_filter, num_kernel,n_timesteps, n_steps, n_length, n_features, n_outputs)
  optimizer = keras.optimizers.Adam(learning_rate=learningRate) 
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  model_checkpoint = ModelCheckpoint('Weight_NewTntData(1st_collected_and_simulated_as_tnt)TDCnnBiGRUSAFCN_TF25_v1_86.hdf5', save_best_only=True, monitor='val_accuracy', mode='max', verbose = 0, save_weights_only=True) 
  callbacks_list = [model_checkpoint] 
  history = model.fit(x=trainX, y=trainy, epochs=epochs, validation_data=(valX, valy), batch_size=batch_size, verbose=verbose, callbacks = callbacks_list)   
  _,accuracy = model.evaluate(testX, testy,batch_size=batch_size, verbose=0) 
  return model, accuracy, history

def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

def run_experiment(num_head, repeats, learningRate, batch_size, num_filter, num_kernel, att_neurons, num_lstm_Neuron, num_fcn_Neuron, trainX, trainy, testX, testy, valX, valy,n_timesteps,n_steps,n_length, n_features, n_outputs, cvscores):
	scores = list()
	for r in range(repeats):  
		model, score, history = evaluate_model(num_head, trainX, trainy, testX, testy, valX, valy ,learningRate, batch_size,num_filter, num_kernel, att_neurons, num_lstm_Neuron, num_fcn_Neuron,n_timesteps, n_steps, n_length,n_features, n_outputs)
		score = score * 100.0
		cvscores.append(score)
		print('>#%d: %.3f' % (r+1, score))
		predY = model.predict(testX)
		report = classification_report(testy.argmax(axis=1),predY.argmax(axis=1),target_names=['0','1','2','3','4','5','6','7','8','9'], digits=4, output_dict=True)
		f1_score = report['macro avg']['f1-score']
		f1scores.append(f1_score)    
    
		plt.plot(history.history['loss'])
		plt.plot(history.history['val_loss'])
		plt.title('Training Loss VS Validation Loss')
		plt.ylabel('Loss')
		plt.xlabel('epoch')
		plt.legend(['train', 'val'], loc='upper left')
		plt.show()
		print('Training Accuracy : ', max(history.history['accuracy']))   
		print('Validation Accuracy : ', max(history.history['val_accuracy']))  
		plt.plot(history.history['accuracy'])
		plt.plot(history.history['val_accuracy'])
		plt.title('Training ACC VS Validation ACC')
		plt.ylabel('Acc')
		plt.xlabel('epoch')
		plt.legend(['train', 'val'], loc='upper left')
		plt.show()  
	# model.summary()  
	# summarize_results(scores)
	return model, cvscores, f1scores

# 7. Loading data and reshaping data

## 7.1 Without multiview and all tst as val

In [ ]:
photodiode_arr = ['6', '1', '2', '3', '9', '8', '7', '4', '5']
trainX, trainy, valX, valy, tstX_new, tsty_new  = load_dataset(photodiode_arr)
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
print(trainX.shape)
print(valX.shape)
print(tstX_new.shape)


In [ ]:
trainX1 = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features)) 
valX1 = valX.reshape((valX.shape[0],n_steps, n_length, n_features)) 
tstX_new1 = tstX_new.reshape((tstX_new.shape[0],n_steps, n_length, n_features)) 

# 8 Running the code

## 8.1 Cold-Start && Within-Subjects

In [ ]:
cvscores = []
f1scores = []

for att_neurons in Attention_Neurons:
  for j in range(len(num_fcn_Neurons)):
    for num_kernel in num_kernels:
      for num_filter in num_filters:  
        for learningRate in learning_rate:     
          for batch_size in Batch_size:
            for i in range(len(num_lstm_Neurons)): 
              print('n_steps : ', n_steps)
              print('n_length : ', n_length)
              print('batch_size :  ', batch_size)
              print('learningRate :  ', learningRate)
              print('num_filter :  ', num_filter)
              print('num_kernel :  ', num_kernel)
              print('num_fcn_Neurons :  ', num_fcn_Neurons[j])
              print('num_lstm_neurons :  ', num_lstm_Neurons[i]) 
              print('num_Att_neurons : ', att_neurons)          
              prev_time = time.time()
              model, cvscores, f1scores = run_experiment(num_head = num_head, repeats=10, learningRate= learningRate, batch_size = batch_size, num_filter= num_filter, num_kernel = num_kernel, att_neurons = att_neurons, num_lstm_Neuron = num_lstm_Neurons[i], num_fcn_Neuron = num_lstm_Neurons[i],trainX= trainX1, trainy=trainy, testX=tstX_new1, testy= tsty_new, valX=valX1, valy= valy, n_timesteps = n_timesteps, n_steps=n_steps, n_length = n_length, n_features=n_features, n_outputs=n_outputs, cvscores = cvscores)
              print('Interval is : ', time.time() - prev_time)
              print('-----------------------------------------')
print('cvscores : ', cvscores)
print('avg : ', np.mean(cvscores))
print('f1scores : ', f1scores)
print('avg : ', np.mean(f1scores))

## 8.2 Between-Subjects

In [ ]:
# scikit-learn k-fold cross-validation
from numpy import array
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd 
cvscores = []
kfold = KFold(3, True, 1)
TesterName = ['HaozheTang','Qing','Serge','AlexCui','Keyarash','PingweiJiao','Talia', 'Vineet', 'Song'
        ,'Vito', 'YangLi', 'HanTing','Naram','Ricardo','Weichuan', 'jundaWu','shiXun','XinyunXu',
        'xie','shuxinChen','ZehangWu','Liang','TingLiang','HaoLiu']
for num_digit in range (1):
  photodiode_arr = ['6', '1', '2', '3', '9', '8', '7', '4', '5']
  print('arr : ', photodiode_arr)
  n_features = len(photodiode_arr)
  cvscores = []
  f1scores = []
  for train, test in kfold.split(TesterName):
      trainName = [TesterName[i] for i in train]
      testName = [TesterName[i] for i in test]    
      print('Train : ', trainName)
      print('Test : ', testName)
      trainX, trainy, valX, valy, tstX_new, tsty_new = load_dataset(data_test_filenames_new, label_test_filenames_new, trainName, testName, data_filenames, label_filenames, photodiode_arr)
      n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
      print(trainX.shape)
      print(valX.shape)
      print(tstX_new.shape)
      trainX1 = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features)) 
      valX1 = valX.reshape((valX.shape[0],n_steps, n_length, n_features)) 
      tstX_new1 = tstX_new.reshape((tstX_new.shape[0],n_steps, n_length, n_features)) 
      for num_head in num_heads:
          for att_neurons in Attention_Neurons:
              for j in range(len(num_fcn_Neurons)):
                  for num_kernel in num_kernels:
                      for num_filter in num_filters:  
                          for learningRate in learning_rate:     
                              for batch_size in Batch_size:
                                  for i in range(len(num_lstm_Neurons)): 
                                      print('n_steps : ', n_steps)
                                      print('n_length : ', n_length)
                                      print('batch_size :  ', batch_size)
                                      print('learningRate :  ', learningRate)
                                      print('num_filter :  ', num_filter)
                                      print('num_kernel :  ', num_kernel)
                                      print('num_fcn_Neurons :  ', num_fcn_Neurons[j])
                                      print('num_lstm_neurons :  ', num_lstm_Neurons[i]) 
                                      print('num_heads : ', num_head)
                                      print('num_Att_neurons : ', att_neurons)          
                                      prev_time = time.time()
                                      model, cvscores, f1scores = run_experiment(num_head = num_head, repeats=3, learningRate= learningRate, batch_size = batch_size, num_filter= num_filter, num_kernel = num_kernel, att_neurons = att_neurons, num_lstm_Neuron = num_lstm_Neurons[i], num_fcn_Neuron = num_lstm_Neurons[i],trainX= trainX1, trainy=trainy, testX=tstX_new1, testy= tsty_new, valX=valX1, valy= valy, n_timesteps = n_timesteps, n_steps=n_steps, n_length = n_length, n_features=n_features, n_outputs=n_outputs, cvscores = cvscores)
                                      print('Interval is : ', time.time() - prev_time)
                                      print('-----------------------------------------')
  print('cvscores : ', cvscores)
  print('avg : ', np.mean(cvscores))
  print('f1scores : ', f1scores)
  print('avg : ', np.mean(f1scores))